
# 🔬 UX Analyzer V24.0

### Инструкция:
1. Нажмите "Среда выполнения" → "Выполнить все"
2. Подождите 30 секунд
3. Используйте интерфейс внизу

❗ Не изменяйте ячейки ниже

In [ ]:
#@title { display-mode: "form" }


# -*- coding: utf-8 -*-
"""UX Анализатор V24.0 - Точный анализ с фокусом на бриф"""

# ========================================================================
# 🚀 UX АНАЛИЗАТОР V24.0 - GEMINI EDITION
# ========================================================================
# Изменения v24.0:
# - Удалены все демо данные
# - Усилен фокус на бриф и его вопросы
# - Добавлены ответы на все вопросы брифа
# - Увеличено количество цитат во всех разделах
# - Добавлена проверка достижения целей исследования
# - Результаты стали более конкретными и детальными
# ========================================================================

import subprocess
import sys

# Автоматическая установка зависимостей
print("🚀 Проверка и установка зависимостей...")
try:
    import google.generativeai as genai
    import requests
    import yaml
    import pandas
    import docx
    import matplotlib
    import seaborn
    import wordcloud
    import plotly
    import nltk
    import tqdm
    from weasyprint import HTML as WeasyHTML
    print("✅ Зависимости уже установлены")
except ImportError:
    print("📦 Установка необходимых библиотек...")
    # Системные библиотеки для WeasyPrint
    subprocess.run(['apt-get', 'update', '-qq'], capture_output=True)
    subprocess.run(['apt-get', 'install', '-y', '-qq', 'libpango-1.0-0', 'libpangoft2-1.0-0',
                    'libgdk-pixbuf2.0-0', 'libffi-dev', 'fonts-liberation'], capture_output=True)

    # Python пакеты
    packages = ['google-generativeai', 'requests', 'pyyaml', 'weasyprint', 'pandas', 'python-docx',
                'openpyxl', 'xlrd', 'matplotlib', 'seaborn', 'wordcloud', 'plotly', 'kaleido',
                'tqdm', 'nltk']
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
    print("✅ Все зависимости установлены!")

# Импорты
import os
import json
import re
import time
import logging
import yaml
from datetime import datetime
from typing import Dict, List, Optional, Union, Tuple, Any
from dataclasses import dataclass, field
from pathlib import Path
import traceback
import base64
from collections import defaultdict
import concurrent.futures
from io import BytesIO
import numpy as np
import random
import hashlib
import pickle
from tqdm.notebook import tqdm

import google.generativeai as genai
import requests
import pandas as pd
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from weasyprint import HTML as WeasyHTML
from google.colab import files
from IPython.display import display, HTML, clear_output, FileLink
import ipywidgets as widgets

import nltk
nltk.download('vader_lexicon', quiet=True)
from nltk.sentiment import SentimentIntensityAnalyzer

# Визуализации
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

# Настройка стилей
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.facecolor'] = 'white'
pio.templates.default = "plotly_white"

# Настройка логирования
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ========================================================================
# КОНФИГУРАЦИЯ
# ========================================================================
config_str = """
api:
  gemini:
    model: "gemini-1.5-pro"
    temperature: 0.0
    max_output_tokens: 8192
analysis:
  window_size: 10000
  overlap: 2000
  max_retries: 5
  retry_delay: 5
  min_interviews_recommended: 8
  use_speaker_splitting: true
  require_exact_quotes: true
  min_quote_length: 50
output:
  formats: ['html', 'pdf', 'docx']
"""
config = yaml.safe_load(config_str)

# ========================================================================
# КЛАСС ДЛЯ УПРАВЛЕНИЯ БРИФОМ
# ========================================================================
class BriefManager:
    """Класс для управления брифом исследования"""

    def __init__(self):
        self.brief_data = {
            'research_goals': [],
            'research_questions': [],
            'target_audience': '',
            'business_context': '',
            'success_metrics': [],
            'constraints': []
        }
        self.has_brief = False

    def load_brief(self, content: str):
        """Загрузка и парсинг брифа"""
        self.has_brief = True

        # Простой парсер для текстового брифа
        lines = content.strip().split('\n')
        current_section = None

        section_markers = {
            'цели': 'research_goals',
            'goals': 'research_goals',
            'вопросы': 'research_questions',
            'questions': 'research_questions',
            'аудитория': 'target_audience',
            'audience': 'target_audience',
            'контекст': 'business_context',
            'context': 'business_context',
            'метрики': 'success_metrics',
            'metrics': 'success_metrics',
            'ограничения': 'constraints',
            'constraints': 'constraints'
        }

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # Проверяем, не начало ли это новой секции
            line_lower = line.lower()
            for marker, section in section_markers.items():
                if marker in line_lower:
                    current_section = section
                    break
            else:
                # Это контент секции
                if current_section:
                    if current_section == 'target_audience' or current_section == 'business_context':
                        # Для этих секций сохраняем как строку
                        if self.brief_data[current_section]:
                            self.brief_data[current_section] += ' ' + line
                        else:
                            self.brief_data[current_section] = line
                    else:
                        # Для остальных - как список
                        if line.startswith('-') or line.startswith('•') or line.startswith('*'):
                            line = line[1:].strip()
                        if line:
                            self.brief_data[current_section].append(line)

    def get_brief_context(self):
        """Получение контекста брифа для промптов"""
        if not self.has_brief:
            return ""

        context = "<research_context>\n"
        context += "КРИТИЧЕСКИ ВАЖНО: Все выводы должны отвечать на вопросы и достигать целей из этого брифа!\n\n"

        if self.brief_data['research_goals']:
            context += f"ЦЕЛИ ИССЛЕДОВАНИЯ (ОБЯЗАТЕЛЬНО достичь каждую):\n"
            for i, goal in enumerate(self.brief_data['research_goals'], 1):
                context += f"{i}. {goal}\n"

        if self.brief_data['research_questions']:
            context += f"\nИССЛЕДОВАТЕЛЬСКИЕ ВОПРОСЫ (ОБЯЗАТЕЛЬНО ответить на каждый):\n"
            for i, question in enumerate(self.brief_data['research_questions'], 1):
                context += f"{i}. {question}\n"

        if self.brief_data['target_audience']:
            context += f"\nЦЕЛЕВАЯ АУДИТОРИЯ:\n{self.brief_data['target_audience']}\n"

        if self.brief_data['business_context']:
            context += f"\nБИЗНЕС-КОНТЕКСТ:\n{self.brief_data['business_context']}\n"

        if self.brief_data['success_metrics']:
            context += f"\nМЕТРИКИ УСПЕХА (оценить влияние на каждую):\n"
            for metric in self.brief_data['success_metrics']:
                context += f"- {metric}\n"

        context += "\nВАЖНО: Каждый вывод должен быть подкреплен ТОЧНЫМИ ЦИТАТАМИ из интервью!\n"
        context += "</research_context>\n\n"

        return context

    def get_questions_for_analysis(self):
        """Получение вопросов для анализа"""
        return self.brief_data['research_questions']

    def get_goals_for_analysis(self):
        """Получение целей для анализа"""
        return self.brief_data['research_goals']

# ========================================================================
# УТИЛИТЫ ДЛЯ ОБРАБОТКИ ПЕРЕГРУЗОК API
# ========================================================================
def retry_on_overload(func):
    """Декоратор для автоматической обработки перегрузок API"""
    def wrapper(*args, **kwargs):
        max_retries = config['analysis']['max_retries']
        base_delay = config['analysis']['retry_delay']

        for attempt in range(max_retries):
            try:
                return func(*args, **kwargs)
            except Exception as e:
                error_str = str(e)

                # Проверяем на перегрузку или rate limit
                if any(err in error_str.lower() for err in ["quota", "rate", "limit", "overload", "429", "503"]):
                    if attempt < max_retries - 1:
                        delay = base_delay * (2 ** attempt) + random.uniform(0, 5)
                        print(f"   ⏳ API перегружен. Ожидание {delay:.1f} сек... (попытка {attempt + 1}/{max_retries})")
                        time.sleep(delay)
                    else:
                        print(f"   ❌ Не удалось выполнить запрос после {max_retries} попыток")
                        raise
                else:
                    raise

        raise Exception(f"Не удалось выполнить запрос после {max_retries} попыток")

    return wrapper

# ========================================================================
# GEMINI API WRAPPER
# ========================================================================
class GeminiAPIWrapper:
    """Обертка для безопасных вызовов Gemini API"""

    def __init__(self, api_key: str):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(config['api']['gemini']['model'])
        self.generation_config = genai.types.GenerationConfig(
            temperature=config['api']['gemini']['temperature'],
            max_output_tokens=config['api']['gemini']['max_output_tokens'],
        )

    @retry_on_overload
    def generate_content(self, prompt: str) -> str:
        """Генерация контента с автоматическими повторами"""
        response = self.model.generate_content(
            prompt,
            generation_config=self.generation_config
        )
        return response.text

# ========================================================================
# ДАТАКЛАССЫ
# ========================================================================
@dataclass
class CompanyConfig:
    name: str = "Company"
    report_title: str = "UX Research Report"
    author: str = "Research Team"

@dataclass
class InterviewSummary:
    """Структура саммари интервью"""
    interview_id: int
    respondent_profile: Dict[str, Any]
    key_themes: List[Dict[str, Any]]
    pain_points: List[Dict[str, Any]]
    needs: List[Dict[str, Any]]
    insights: List[str]
    emotional_journey: List[Dict[str, Any]]
    contradictions: List[str]
    quotes: List[Dict[str, Any]]
    business_pains: List[Dict[str, Any]] = field(default_factory=list)
    user_problems: List[Dict[str, Any]] = field(default_factory=list)
    opportunities: List[str] = field(default_factory=list)
    sentiment_score: float = field(default=0.0)
    brief_related_findings: Dict[str, Any] = field(default_factory=dict)

@dataclass
class ResearchFindings:
    """Структура результатов исследования"""
    executive_summary: str
    key_insights: List[Dict[str, Any]]
    behavioral_patterns: List[Dict[str, Any]]
    user_segments: List[Dict[str, Any]]
    pain_points_map: Dict[str, List[Dict[str, Any]]]
    opportunities: List[Dict[str, Any]]
    recommendations: List[Dict[str, Any]]
    risks: List[Dict[str, Any]]
    personas: List[Dict[str, Any]]
    current_metrics: Dict[str, Any] = field(default_factory=dict)
    brief_answers: Dict[str, Any] = field(default_factory=dict)
    goal_achievement: Dict[str, Any] = field(default_factory=dict)

# ========================================================================
# КЛАСС ДЛЯ КЭШИРОВАНИЯ
# ========================================================================
class CacheManager:
    def __init__(self, cache_dir="cache"):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)

    def get_hash(self, data: str) -> str:
        """Генерация хеша для данных"""
        return hashlib.md5(data.encode()).hexdigest()

    def get(self, key: str):
        """Получить из кэша"""
        cache_file = self.cache_dir / f"{key}.pkl"
        if cache_file.exists():
            with open(cache_file, 'rb') as f:
                return pickle.load(f)
        return None

    def set(self, key: str, value: Any):
        """Сохранить в кэш"""
        cache_file = self.cache_dir / f"{key}.pkl"
        with open(cache_file, 'wb') as f:
            pickle.dump(value, f)

# ========================================================================
# УЛУЧШЕННЫЙ КЛАСС ДЛЯ АНАЛИЗА С GEMINI
# ========================================================================
class AdvancedGeminiAnalyzer:
    def __init__(self, api_key: str):
        self.api_wrapper = GeminiAPIWrapper(api_key)
        self.window_size = config['analysis']['window_size']
        self.overlap = config['analysis']['overlap']
        self.interview_summaries = []
        self.cache = CacheManager()
        self.brief_manager = BriefManager()

    def set_brief(self, brief_content: str):
        """Установка брифа исследования"""
        self.brief_manager.load_brief(brief_content)

    def analyze_transcripts_parallel(self, transcripts: List[str]) -> Dict:
        """Анализ с параллельной обработкой"""
        print("🧠 Начинаю параллельный анализ...")

        # Проверка количества интервью
        if len(transcripts) < config['analysis']['min_interviews_recommended']:
            print(f"⚠️  ВНИМАНИЕ: Рекомендуется минимум {config['analysis']['min_interviews_recommended']} интервью для качественного анализа!")
            print(f"   У вас: {len(transcripts)} интервью")
            print("   Результаты могут быть недостаточно репрезентативными\n")

        # Ограничиваем количество параллельных запросов
        max_workers = min(3, len(transcripts))

        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_idx = {
                executor.submit(self._deep_analyze_interview, transcript, i+1): i
                for i, transcript in enumerate(transcripts)
            }

            interview_summaries = [None] * len(transcripts)

            for future in tqdm(concurrent.futures.as_completed(future_to_idx),
                              total=len(transcripts), desc="Анализ интервью"):
                idx = future_to_idx[future]
                try:
                    interview_summaries[idx] = future.result()
                except Exception as e:
                    print(f"❌ Ошибка при анализе интервью {idx+1}: {e}")
                    interview_summaries[idx] = self._create_empty_summary(idx+1)

        interview_summaries = [s for s in interview_summaries if s is not None]
        self.interview_summaries = interview_summaries

        return self._continue_analysis(interview_summaries, len(transcripts))

    def analyze_transcripts(self, transcripts: List[str]) -> Dict:
        """Комплексный анализ транскриптов с прогресс-барами"""
        print("🧠 Начинаю глубокий анализ...")

        # Проверка количества интервью
        if len(transcripts) < config['analysis']['min_interviews_recommended']:
            print(f"⚠️  ВНИМАНИЕ: Рекомендуется минимум {config['analysis']['min_interviews_recommended']} интервью!")
            print(f"   У вас: {len(transcripts)} интервью")

        with tqdm(total=12, desc="Общий прогресс") as pbar:
            pbar.set_description("Анализ интервью")
            interview_summaries = []

            for i, transcript in enumerate(tqdm(transcripts, desc="Интервью", leave=False)):
                summary = self._deep_analyze_interview(transcript, i+1)
                interview_summaries.append(summary)

            self.interview_summaries = interview_summaries
            pbar.update(1)

        return self._continue_analysis(interview_summaries, len(transcripts))

    def _continue_analysis(self, interview_summaries: List[InterviewSummary], total_interviews: int) -> Dict:
        """Продолжение анализа после обработки интервью"""
        with tqdm(total=12, desc="Общий прогресс", initial=1) as pbar:
            # 2. Генерация текущих метрик
            pbar.set_description("Генерация метрик")
            current_metrics = self._generate_current_metrics(interview_summaries)
            pbar.update(1)

            # 3. Кросс-анализ интервью
            pbar.set_description("Кросс-анализ")
            cross_analysis = self._cross_analyze_interviews(interview_summaries)
            pbar.update(1)

            # 4. Дедупликация болей
            pbar.set_description("Дедупликация болей")
            deduplicated_pains = self._deduplicate_pains(interview_summaries)
            pbar.update(1)

            # 5. Выявление поведенческих паттернов
            pbar.set_description("Поиск паттернов")
            patterns = self._identify_behavioral_patterns(interview_summaries, cross_analysis)
            pbar.update(1)

            # 6. Сегментация аудитории
            pbar.set_description("Сегментация")
            segments = self._segment_audience(interview_summaries, patterns)
            pbar.update(1)

            # 7. Создание персон
            pbar.set_description("Создание персон")
            personas = self._create_personas(segments, interview_summaries)
            pbar.update(1)

            # 8. Генерация инсайтов и рекомендаций
            pbar.set_description("Генерация инсайтов")
            findings = self._generate_final_findings(
                interview_summaries, cross_analysis, patterns, segments, personas
            )
            pbar.update(1)

            findings.current_metrics = current_metrics

            # 9. Генерация рекомендаций
            pbar.set_description("Генерация рекомендаций")
            recommendations = self._generate_recommendations(findings.key_insights)
            pbar.update(1)

            # 10. Генерация материалов для защиты
            pbar.set_description("Материалы для защиты")
            defense_materials = self._generate_defense_materials(findings, recommendations, total_interviews)
            pbar.update(1)

            # 11. Ответы на вопросы брифа
            pbar.set_description("Ответы на вопросы брифа")
            brief_answers = self._analyze_brief_questions(interview_summaries, findings)
            findings.brief_answers = brief_answers
            pbar.update(1)

            # 12. Оценка достижения целей
            pbar.set_description("Оценка достижения целей")
            goal_achievement = self._assess_goal_achievement(findings, interview_summaries)
            findings.goal_achievement = goal_achievement
            pbar.update(1)

        return {
            'base_analysis': {
                'segments': segments,
                'problems': findings.key_insights,
                'insights': self._format_insights_for_report(findings.key_insights),
                'user_journey_issues': []
            },
            'recommendations': recommendations,
            'defense_materials': defense_materials,
            'interview_summaries': interview_summaries,
            'findings': findings,
            'total_interviews': total_interviews,
            'current_metrics': current_metrics,
            'personas': personas,
            'brief_data': self.brief_manager.brief_data if self.brief_manager.has_brief else None,
            'brief_answers': brief_answers,
            'goal_achievement': goal_achievement
        }

    def _detect_speaker_format(self, text: str) -> bool:
        """Определение наличия диаризации"""
        patterns = [
            r'^(Speaker\s*\d+|Спикер\s*\d+|Interviewer|Интервьюер|Respondent|Респондент)[:：]\s*',
            r'^([А-ЯA-Z][а-яa-z]+\s*[А-ЯA-Z]?\.?\s*)[:：]\s*',
            r'^\[([^\]]+)\][:：]?\s*',
            r'^-\s*([А-ЯA-Z][а-яa-z]+)[:：]\s*',
        ]

        lines = text.split('\n')
        speaker_lines = 0

        for line in lines[:50]:
            line = line.strip()
            if not line:
                continue

            for pattern in patterns:
                if re.match(pattern, line, re.MULTILINE | re.IGNORECASE):
                    speaker_lines += 1
                    break

        return speaker_lines > len([l for l in lines[:50] if l.strip()]) * 0.3

    def _split_by_speakers(self, text: str) -> List[Dict[str, str]]:
        """Разделение текста по спикерам"""
        patterns = [
            r'^(Speaker\s*\d+|Спикер\s*\d+|Interviewer|Интервьюер|Respondent|Респондент)[:：]\s*(.+)',
            r'^([А-ЯA-Z][а-яa-z]+\s*[А-ЯA-Z]?\.?\s*)[:：]\s*(.+)',
            r'^\[([^\]]+)\][:：]?\s*(.+)',
            r'^-\s*([А-ЯA-Z][а-яa-z]+)[:：]\s*(.+)',
        ]

        segments = []
        current_speaker = "Unknown"
        current_text = []

        lines = text.split('\n')

        for line in lines:
            line = line.strip()
            if not line:
                continue

            speaker_found = False
            for pattern in patterns:
                match = re.match(pattern, line, re.MULTILINE | re.IGNORECASE)
                if match:
                    if current_text:
                        segments.append({
                            'speaker': current_speaker,
                            'text': ' '.join(current_text)
                        })

                    current_speaker = match.group(1).strip()
                    current_text = [match.group(2).strip()]
                    speaker_found = True
                    break

            if not speaker_found:
                current_text.append(line)

        if current_text:
            segments.append({
                'speaker': current_speaker,
                'text': ' '.join(current_text)
            })

        return segments

    def _create_speaker_based_chunks(self, text: str) -> List[str]:
        """Создание чанков на основе реплик спикеров"""
        segments = self._split_by_speakers(text)

        if not segments:
            return self._create_overlapping_chunks(text)

        chunks = []
        current_chunk = []
        current_size = 0

        for segment in segments:
            segment_text = f"{segment['speaker']}: {segment['text']}"
            segment_size = len(segment_text)

            if current_size + segment_size > self.window_size and current_chunk:
                chunks.append('\n'.join(current_chunk))

                overlap_segments = []
                overlap_size = 0
                for i in range(len(current_chunk) - 1, -1, -1):
                    seg_size = len(current_chunk[i])
                    if overlap_size + seg_size <= self.overlap:
                        overlap_segments.insert(0, current_chunk[i])
                        overlap_size += seg_size
                    else:
                        break

                current_chunk = overlap_segments + [segment_text]
                current_size = overlap_size + segment_size
            else:
                current_chunk.append(segment_text)
                current_size += segment_size

        if current_chunk:
            chunks.append('\n'.join(current_chunk))

        return chunks

    def _create_overlapping_chunks(self, text: str) -> List[str]:
        """Создание чанков с перекрытием"""
        chunks = []
        start = 0

        if len(text) <= self.window_size:
            return [text]

        while start < len(text):
            end = start + self.window_size
            chunk = text[start:end]
            chunks.append(chunk)
            start += self.window_size - self.overlap

        return chunks

    def _deep_analyze_interview(self, transcript: str, interview_num: int) -> InterviewSummary:
        """Глубокий анализ одного интервью"""
        cache_key = f"interview_{interview_num}_{self.cache.get_hash(transcript[:1000])}"
        cached = self.cache.get(cache_key)
        if cached:
            print(f"   📦 Используем кэшированный результат для интервью {interview_num}")
            return cached

        full_transcript = transcript

        # Определяем наличие диаризации и создаем чанки
        if config['analysis']['use_speaker_splitting'] and self._detect_speaker_format(full_transcript):
            print(f"   🎤 Обнаружена диаризация для интервью {interview_num}")
            chunks = self._create_speaker_based_chunks(full_transcript)
        else:
            chunks = self._create_overlapping_chunks(full_transcript)

        # Суммаризируем каждый чанк
        chunk_summaries = []
        for i, chunk in enumerate(chunks):
            if len(chunks) > 1:
                print(f"   Анализ части {i+1}/{len(chunks)}...")

            summary = self._summarize_chunk(chunk)
            if summary:
                chunk_summaries.append(summary)

        combined_summary = "\n\n".join(chunk_summaries)

        # Разбиваем анализ на части
        profile_and_themes = self._analyze_profile_and_themes(combined_summary, interview_num)
        pains_and_needs = self._analyze_pains_and_needs(combined_summary, interview_num)
        emotions_and_insights = self._analyze_emotions_and_insights(combined_summary, interview_num)
        quotes_and_contradictions = self._analyze_quotes_and_contradictions(combined_summary, interview_num)
        business_aspects = self._analyze_business_aspects(combined_summary, interview_num)

        # Анализ связанный с брифом
        brief_findings = {}
        if self.brief_manager.has_brief:
            brief_findings = self._analyze_brief_related_content(combined_summary, interview_num)

        # Объединяем результаты
        data = {
            'interview_id': interview_num,
            'respondent_profile': profile_and_themes.get('respondent_profile', {}),
            'key_themes': profile_and_themes.get('key_themes', []),
            'pain_points': pains_and_needs.get('pain_points', []),
            'needs': pains_and_needs.get('needs', []),
            'insights': emotions_and_insights.get('insights', []),
            'emotional_journey': emotions_and_insights.get('emotional_journey', []),
            'contradictions': quotes_and_contradictions.get('contradictions', []),
            'quotes': quotes_and_contradictions.get('quotes', []),
            'business_pains': business_aspects.get('business_pains', []),
            'user_problems': business_aspects.get('user_problems', []),
            'opportunities': business_aspects.get('opportunities', []),
            'brief_related_findings': brief_findings
        }

        # Sentiment analysis
        try:
            sia = SentimentIntensityAnalyzer()
            sentiment_score = sia.polarity_scores(full_transcript)
            data['sentiment_score'] = sentiment_score['compound']
        except:
            data['sentiment_score'] = 0

        result = InterviewSummary(**data)

        # Сохраняем в кэш
        self.cache.set(cache_key, result)

        return result

    @retry_on_overload
    def _analyze_profile_and_themes(self, summary: str, interview_num: int) -> Dict:
        """Анализ профиля респондента и ключевых тем"""
        context = self.brief_manager.get_brief_context()

        prompt = f"""{context}

Ты — ведущий UX-исследователь с 20-летним опытом в качественном анализе данных.

Проанализируй интервью №{interview_num} и извлеки максимально детальную информацию о респонденте и ключевых темах.

КРИТИЧЕСКИ ВАЖНО:
- Используй ТОЛЬКО информацию из интервью
- НЕ придумывай никаких данных
- Каждая тема должна быть подкреплена ТОЧНЫМИ ЦИТАТАМИ (минимум 60 слов)
- Если информация отсутствует, указывай "Не упоминается в интервью"

Верни ТОЛЬКО валидный JSON в следующем формате:
{{
    "respondent_profile": {{
        "demographics": "ТОЛЬКО то, что явно сказано в интервью",
        "occupation": "ТОЛЬКО если упоминается профессия",
        "experience_level": "ТОЛЬКО реальный опыт из интервью",
        "context": "ТОЛЬКО реальный контекст из интервью",
        "tech_literacy": "ТОЛЬКО если есть данные",
        "motivations": "ТОЛЬКО явные мотивации из интервью",
        "lifestyle": "ТОЛЬКО если упоминается",
        "archetype": "На основе РЕАЛЬНЫХ данных",
        "unique_traits": "ТОЛЬКО уникальные черты из интервью"
    }},
    "key_themes": [
        {{
            "theme": "Название темы",
            "description": "Детальное описание темы",
            "frequency": "Сколько раз упоминалась",
            "importance": "Важность для респондента",
            "quotes": ["ПОЛНАЯ цитата минимум 60 слов", "Еще одна ПОЛНАЯ цитата"],
            "emotional_tone": "Эмоциональный окрас темы",
            "relevance_to_brief": "Как связано с целями брифа"
        }}
    ]
}}

СУММАРИ ИНТЕРВЬЮ:
{summary[:4000]}"""

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _analyze_pains_and_needs(self, summary: str, interview_num: int) -> Dict:
        """Анализ болей и потребностей"""
        context = self.brief_manager.get_brief_context()

        prompt = f"""{context}

Ты — эксперт по выявлению пользовательских проблем и скрытых потребностей.

Проанализируй интервью №{interview_num} и найди ВСЕ боли, проблемы и потребности респондента.

КРИТИЧЕСКИ ВАЖНО:
1. КАЖДАЯ боль должна быть подкреплена ТОЧНОЙ ЦИТАТОЙ (минимум 60 слов)
2. НЕ придумывай проблемы - только то, что ЯВНО сказано
3. Связывай каждую находку с целями и вопросами брифа
4. Приводи ВСЕ доказательства из интервью

Верни ТОЛЬКО валидный JSON:
{{
    "pain_points": [
        {{
            "pain": "ТОЧНОЕ описание боли из интервью",
            "pain_type": "functional/process/emotional/social/financial",
            "root_cause": "Корневая причина из интервью",
            "symptoms": ["Симптом из интервью", "Еще симптом"],
            "context": "ТОЧНЫЙ контекст из интервью",
            "severity": "critical/high/medium/low",
            "frequency": "ТОЧНАЯ частота из интервью",
            "impact": "ТОЧНОЕ влияние из слов респондента",
            "current_solution": "Что ТОЧНО делает сейчас",
            "ideal_solution": "Что ТОЧНО хочет",
            "quotes": ["ПОЛНАЯ цитата о проблеме минимум 60 слов"],
            "emotional_impact": "ТОЧНЫЕ эмоции из интервью",
            "relevance_to_brief": "Как связано с вопросами брифа"
        }}
    ],
    "needs": [
        {{
            "need": "ТОЧНАЯ формулировка потребности",
            "need_type": "functional/emotional/social/self-actualization",
            "job_to_be_done": "Что ТОЧНО пытается сделать",
            "current_satisfaction": "Насколько удовлетворена по словам респондента",
            "importance": "critical/high/medium/low",
            "triggers": ["ТОЧНЫЙ триггер из интервью"],
            "barriers": ["ТОЧНЫЙ барьер из интервью"],
            "success_criteria": "Что будет успехом по словам респондента",
            "quotes": ["ПОЛНАЯ цитата о потребности минимум 60 слов"],
            "related_pains": ["Связанные боли"],
            "relevance_to_brief": "Как отвечает на вопросы брифа"
        }}
    ]
}}

СУММАРИ:
{summary[:4000]}"""

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _analyze_emotions_and_insights(self, summary: str, interview_num: int) -> Dict:
        """Глубокий анализ эмоций и инсайтов"""
        context = self.brief_manager.get_brief_context()

        prompt = f"""{context}

Ты — эксперт по эмоциональному дизайну и поведенческой психологии.

Проведи глубочайший анализ эмоционального опыта респондента в интервью №{interview_num}.

КРИТИЧЕСКИ ВАЖНО:
1. КАЖДЫЙ инсайт должен быть основан на ТОЧНЫХ ЦИТАТАХ (минимум 80 слов)
2. НЕ придумывай эмоции - только то, что ЯВНО выражено
3. Каждый вывод должен помогать ответить на вопросы брифа
4. Приводи ВСЕ доказательства и цитаты

Верни ТОЛЬКО валидный JSON:
{{
    "emotional_journey": [
        {{
            "moment": "ТОЧНОЕ описание момента из интервью",
            "trigger": "Что ТОЧНО вызвало эмоцию",
            "emotion": "ТОЧНОЕ название эмоции из контекста",
            "emotion_family": "primary/secondary/social/cognitive",
            "intensity": 8,
            "valence": "positive/negative/mixed",
            "duration": "Длительность если упоминается",
            "body_language": "ТОЛЬКО если описано в интервью",
            "quote": "ПОЛНАЯ цитата минимум 80 слов",
            "coping": "Как справлялся ПО СЛОВАМ респондента",
            "impact": "Влияние ПО СЛОВАМ респондента",
            "underlying_need": "Потребность из контекста",
            "relevance_to_brief": "Связь с целями исследования"
        }}
    ],
    "emotional_patterns": [
        {{
            "pattern": "Название паттерна из данных",
            "description": "Детальное описание на основе интервью",
            "triggers": ["Триггер из интервью"],
            "manifestation": "Как проявляется по данным",
            "frequency": "Частота из интервью",
            "coping_strategies": ["Стратегия из интервью"],
            "design_implications": "Выводы для дизайна",
            "quotes": ["Подтверждающая цитата минимум 60 слов"]
        }}
    ],
    "insights": [
        {{
            "insight": "Глубокий инсайт основанный на данных (минимум 80 слов)",
            "insight_type": "behavioral/emotional/cognitive/motivational",
            "confidence": "high/medium/low",
            "evidence": [
                "ТОЧНОЕ доказательство из интервью",
                "Еще доказательство",
                "Третье доказательство"
            ],
            "contradiction": "Противоречие если есть",
            "hidden_motivation": "Скрытая мотивация из контекста",
            "design_opportunity": "Возможность для дизайна",
            "business_impact": "Влияние на бизнес",
            "quotes": ["ПОЛНАЯ подтверждающая цитата минимум 80 слов", "Еще цитата"],
            "relevance_to_brief": "Как помогает достичь целей исследования"
        }}
    ],
    "cognitive_biases": [
        {{
            "bias": "Название искажения",
            "manifestation": "Как проявляется В ДАННОМ интервью",
            "impact": "Влияние по данным интервью",
            "design_consideration": "Как учесть",
            "quotes": ["Подтверждающая цитата"]
        }}
    ]
}}

СУММАРИ:
{summary}"""

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _analyze_quotes_and_contradictions(self, summary: str, interview_num: int) -> Dict:
        """Анализ важных цитат и противоречий"""
        context = self.brief_manager.get_brief_context()

        prompt = f"""{context}

Ты — эксперт по дискурс-анализу и семантическому анализу текста.

Найди самые важные цитаты и ВСЕ противоречия в интервью №{interview_num}.

КРИТИЧЕСКИ ВАЖНО:
1. Приводи ТОЛЬКО ПОЛНЫЕ ТОЧНЫЕ цитаты (минимум 80 слов)
2. НЕ сокращай и НЕ перефразируй
3. Каждая цитата должна быть ДОСЛОВНОЙ
4. Выбирай цитаты, которые отвечают на вопросы брифа

Верни ТОЛЬКО валидный JSON:
{{
    "power_quotes": [
        {{
            "quote_id": "Q1",
            "text": "ПОЛНАЯ ДОСЛОВНАЯ цитата респондента (минимум 80 слов)",
            "context": "Детальный контекст высказывания",
            "significance": "Почему эта цитата критически важна для исследования",
            "reveals": {{
                "about_user": "Что ТОЧНО раскрывает о пользователе",
                "about_product": "Что ТОЧНО говорит о продукте",
                "about_market": "Что показывает о рынке"
            }},
            "emotions": ["Эмоция из контекста"],
            "keywords": ["Ключевое слово из цитаты"],
            "metaphors": ["Метафора если есть в цитате"],
            "quote_type": "pain/need/insight/emotion/solution",
            "usability": "Как использовать для достижения целей брифа",
            "relevance_to_questions": "К каким вопросам брифа относится"
        }}
    ],
    "contradictions": [
        {{
            "contradiction_type": "logical/emotional/behavioral/temporal/value",
            "severity": "high/medium/low",
            "statement_1": {{
                "text": "ТОЧНОЕ первое утверждение",
                "context": "Контекст утверждения",
                "emotional_state": "Эмоциональное состояние"
            }},
            "statement_2": {{
                "text": "ТОЧНОЕ противоречащее утверждение",
                "context": "Контекст",
                "emotional_state": "Эмоциональное состояние"
            }},
            "analysis": {{
                "nature": "В чем ТОЧНО суть противоречия",
                "possible_reasons": ["Возможная причина из контекста"],
                "underlying_conflict": "Глубинный конфликт",
                "resolution_attempts": "Попытки разрешения если есть"
            }},
            "implications": {{
                "for_design": "Что значит для дизайна",
                "for_research": "Что значит для целей исследования"
            }},
            "full_quotes": ["Полная цитата с противоречием 1", "Полная цитата 2"]
        }}
    ],
    "language_patterns": [
        {{
            "pattern": "Языковой паттерн из интервью",
            "frequency": "Точная частота использования",
            "meaning": "Что означает в контексте",
            "emotional_load": "Эмоциональная нагрузка",
            "examples": ["Пример использования из интервью"]
        }}
    ]
}}

СУММАРИ:
{summary}"""

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _analyze_business_aspects(self, summary: str, interview_num: int) -> Dict:
        """Анализ бизнес-аспектов и возможностей"""
        context = self.brief_manager.get_brief_context()

        prompt = f"""{context}

Ты — стратегический консультант по продуктам с экспертизой в UX и бизнес-метриках.

Проанализируй бизнес-влияние проблем из интервью №{interview_num} и найди возможности для роста.

КРИТИЧЕСКИ ВАЖНО:
1. Основывайся ТОЛЬКО на реальных данных из интервью
2. Каждый вывод подкрепляй ТОЧНЫМИ ЦИТАТАМИ
3. Связывай с метриками успеха из брифа
4. НЕ придумывай возможности - только из данных

Верни ТОЛЬКО валидный JSON:
{{
    "business_pains": [
        {{
            "pain": "Бизнес-проблема из данных интервью",
            "source": "ТОЧНАЯ пользовательская проблема-источник",
            "impact": {{
                "revenue": "Влияние если упоминается",
                "costs": "Влияние если упоминается",
                "efficiency": "Влияние если упоминается",
                "reputation": "Влияние если упоминается"
            }},
            "affected_metrics": ["Метрика из брифа если релевантна"],
            "quantification": "Количественная оценка ЕСЛИ ЕСТЬ в интервью",
            "urgency": "critical/high/medium/low",
            "dependencies": ["Зависимость из интервью"],
            "quotes": ["ПОЛНАЯ подтверждающая цитата минимум 60 слов"],
            "relevance_to_success_metrics": "Связь с метриками успеха из брифа"
        }}
    ],
    "user_problems": [
        {{
            "problem": "ТОЧНАЯ проблема пользователя",
            "jobs_to_be_done": "Что ТОЧНО не может сделать",
            "frequency": "ТОЧНАЯ частота из интервью",
            "severity": "blocker/major/minor",
            "workaround": "ТОЧНОЕ текущее решение",
            "workaround_cost": "Цена решения если упоминается",
            "segments_affected": ["Сегмент из данных"],
            "competitive_advantage": "Преимущество если решить",
            "solution_criteria": ["Критерий из интервью"],
            "quotes": ["ПОЛНАЯ цитата о проблеме минимум 60 слов"],
            "impact_on_goals": "Как влияет на достижение целей брифа"
        }}
    ],
    "opportunities": [
        {{
            "opportunity": "Возможность ОСНОВАННАЯ на данных интервью",
            "opportunity_type": "quick_win/strategic/innovation/optimization",
            "based_on_problems": ["Проблема из интервью"],
            "value_proposition": "Ценность из контекста интервью",
            "target_segments": ["Сегмент из данных"],
            "implementation": {{
                "complexity": "low/medium/high",
                "timeline": "Оценка если возможна",
                "resources": "Ресурсы если обсуждались",
                "risks": ["Риск если упоминался"]
            }},
            "expected_impact": {{
                "user_value": "Ценность из слов пользователя",
                "business_value": "Ценность для бизнеса",
                "metrics": {{
                    "metric_name": "изменение если можно оценить"
                }}
            }},
            "success_criteria": ["Критерий из интервью"],
            "quotes": ["ПОЛНАЯ поддерживающая цитата минимум 60 слов"],
            "alignment_with_brief": "Как помогает достичь целей брифа"
        }}
    ]
}}

СУММАРИ:
{summary}"""

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _analyze_brief_related_content(self, summary: str, interview_num: int) -> Dict:
        """Анализ контента связанного с брифом"""
        context = self.brief_manager.get_brief_context()
        questions = self.brief_manager.get_questions_for_analysis()
        goals = self.brief_manager.get_goals_for_analysis()

        prompt = f"""{context}

Найди в интервью №{interview_num} ВСЕ упоминания и данные, относящиеся к целям и вопросам брифа.

ЦЕЛИ ИССЛЕДОВАНИЯ:
{json.dumps(goals, ensure_ascii=False)}

ВОПРОСЫ ИССЛЕДОВАНИЯ:
{json.dumps(questions, ensure_ascii=False)}

Для КАЖДОЙ цели и КАЖДОГО вопроса найди:
1. Прямые ответы и упоминания
2. Косвенные данные и инсайты
3. ТОЧНЫЕ ЦИТАТЫ (минимум 60 слов)

Верни JSON:
{{
    "goal_related_findings": [
        {{
            "goal": "Цель из брифа",
            "findings": [
                {{
                    "finding": "Что найдено в интервью",
                    "quote": "ПОЛНАЯ цитата минимум 60 слов",
                    "relevance": "Как относится к цели",
                    "strength": "strong/moderate/weak"
                }}
            ]
        }}
    ],
    "question_related_findings": [
        {{
            "question": "Вопрос из брифа",
            "answers": [
                {{
                    "answer": "Ответ из интервью",
                    "quote": "ПОЛНАЯ цитата минимум 60 слов",
                    "confidence": "high/medium/low",
                    "additional_context": "Дополнительный контекст"
                }}
            ]
        }}
    ],
    "metric_related_findings": [
        {{
            "metric": "Метрика из брифа",
            "current_state": "Текущее состояние по данным",
            "user_perception": "Восприятие пользователя",
            "improvement_suggestions": ["Предложение из интервью"],
            "quotes": ["Подтверждающая цитата"]
        }}
    ]
}}

СУММАРИ:
{summary}"""

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    def _create_empty_summary(self, interview_num: int) -> InterviewSummary:
        """Создание пустого саммари при ошибках"""
        return InterviewSummary(
            interview_id=interview_num,
            respondent_profile={'demographics': 'Ошибка анализа'},
            key_themes=[],
            pain_points=[],
            needs=[],
            insights=['Требуется повторный анализ'],
            emotional_journey=[],
            contradictions=[],
            quotes=[],
            business_pains=[],
            user_problems=[],
            opportunities=[],
            sentiment_score=0.0,
            brief_related_findings={}
        )

    @retry_on_overload
    def _summarize_chunk(self, chunk: str) -> str:
        """Детальная суммаризация чанка"""
        context = self.brief_manager.get_brief_context()

        prompt = f"""{context}

Ты — ведущий UX-исследователь. Проанализируй фрагмент интервью и извлеки ВСЮ ценную информацию.

КРИТИЧЕСКИ ВАЖНО:
1. Сохрани ВСЕ важные цитаты ПОЛНОСТЬЮ и ДОСЛОВНО (минимум 60 слов)
2. НЕ обобщай и НЕ перефразируй - копируй точные формулировки
3. Фиксируй ВСЕ детали: имена, бренды, суммы, даты, проценты
4. Отмечай ВСЕ эмоциональные реакции
5. Связывай находки с целями и вопросами брифа

СТРУКТУРА АНАЛИЗА:

### РЕСПОНДЕНТ
[ВСЯ информация о респонденте из фрагмента БЕЗ додумывания]

### КЛЮЧЕВЫЕ ПРОБЛЕМЫ
Для каждой проблемы:
- Проблема: [ТОЧНОЕ название из интервью]
- Цитата: "[ПОЛНАЯ ДОСЛОВНАЯ цитата респондента минимум 60 слов]"
- Контекст: [ТОЧНЫЕ детали ситуации]
- Эмоции: [ТОЛЬКО упомянутые эмоции]
- Последствия: [ТОЛЬКО сказанное респондентом]
- Попытки решения: [ТОЛЬКО упомянутое]
- Связь с брифом: [К какой цели/вопросу относится]

### ПОТРЕБНОСТИ И ЖЕЛАНИЯ
- Явные потребности: [ТОЛЬКО прямо сказанное]
- Скрытые потребности: [ТОЛЬКО с доказательствами]
- Идеальное решение: [ТОЛЬКО слова респондента]
- Цитаты: "[Каждая цитата ПОЛНОСТЬЮ]"

### ЭМОЦИОНАЛЬНЫЙ КОНТЕКСТ
- Эмоциональные пики: [ТОЧНЫЕ моменты с цитатами]
- Изменения настроения: [С доказательствами]
- Метафоры и образы: [ДОСЛОВНО из интервью]

### ПОВЕДЕНЧЕСКИЕ ПАТТЕРНЫ
- Привычки: [ТОЛЬКО упомянутые]
- Обходные пути: [ТОЧНО как описано]
- Триггеры: [ТОЛЬКО из интервью]
- Цитаты: "[Подтверждающие цитаты]"

### КОНТЕКСТ И ДЕТАЛИ
- Демография: [ТОЛЬКО упоминания]
- Технический уровень: [ТОЛЬКО из контекста]
- Финансовые аспекты: [ТОЛЬКО упомянутые суммы/проценты]
- Временные рамки: [ТОЛЬКО указанные]

### ОТВЕТЫ НА ВОПРОСЫ БРИФА
[Для каждого вопроса брифа - что нашлось в этом фрагменте]

### ИНСАЙТЫ И ПРОТИВОРЕЧИЯ
- Ключевые инсайты: [С полными цитатами]
- Противоречия: [ТОЧНЫЕ несоответствия]
- Моменты озарения: [С контекстом]

ФРАГМЕНТ ИНТЕРВЬЮ:
{chunk}"""

        response = self.api_wrapper.generate_content(prompt)
        return response

    @retry_on_overload
    def _deduplicate_pains(self, interview_summaries: List[InterviewSummary]) -> List[Dict]:
        """Дедупликация болей через LLM"""
        all_pains = []
        for summary in interview_summaries:
            for pain in summary.pain_points:
                all_pains.append({
                    'pain': pain.get('pain', ''),
                    'context': pain.get('context', ''),
                    'interview_id': summary.interview_id,
                    'quotes': pain.get('quotes', []),
                    'severity': pain.get('severity', 'medium'),
                    'impact': pain.get('impact', ''),
                    'frequency': pain.get('frequency', ''),
                    'relevance_to_brief': pain.get('relevance_to_brief', '')
                })

            for pain in summary.business_pains:
                all_pains.append({
                    'pain': pain.get('pain', ''),
                    'context': pain.get('context', ''),
                    'interview_id': summary.interview_id,
                    'quotes': pain.get('quotes', []),
                    'business_impact': pain.get('impact', {}),
                    'relevance_to_brief': pain.get('relevance_to_success_metrics', '')
                })

        if not all_pains:
            return []

        context = self.brief_manager.get_brief_context()

        prompt = f'''{context}

Ты — эксперт по качественному анализу данных.

Проанализируй {len(all_pains)} болей из {len(interview_summaries)} интервью и объедини схожие.

КРИТИЧЕСКИ ВАЖНО:
- Объединяй ТОЛЬКО если боли описывают одну корневую проблему
- Сохраняй ВСЕ цитаты из разных интервью
- Сохраняй ВСЕ нюансы и детали
- Приоритизируй по связи с целями брифа

Верни JSON массив уникальных болей:
[
    {{
        "pain_id": "PAIN_001",
        "pain": "Обобщенное описание боли основанное на данных (мин. 80 слов)",
        "pain_variations": ["Точный вариант из интервью 1", "Точный вариант из интервью 3"],
        "root_cause": "Общая корневая причина из данных",
        "contexts": ["Контекст 1 из интервью", "Контекст 2"],
        "interview_ids": [1, 3, 5],
        "frequency_stats": {{
            "absolute": "5 из 8 респондентов",
            "percentage": "62.5%"
        }},
        "quotes": [
            {{
                "text": "ПОЛНАЯ ДОСЛОВНАЯ цитата (мин. 60 слов)",
                "interview_id": 1,
                "emotion": "Эмоциональный контекст"
            }}
        ],
        "severity": {{
            "range": "medium to critical",
            "distribution": {{"critical": 2, "high": 2, "medium": 1}},
            "average": "high"
        }},
        "impact": {{
            "on_users": ["Точное влияние из интервью"],
            "on_business": ["Бизнес-влияние из данных"],
            "time_waste": "Если упоминается",
            "emotional": ["Эмоции из интервью"]
        }},
        "current_workarounds": ["Обходной путь из интервью"],
        "priority_score": 85,
        "priority_reasoning": "Обоснование на основе целей брифа",
        "relevance_to_brief": {{
            "goals": ["Релевантная цель 1"],
            "questions": ["Релевантный вопрос 1"],
            "metrics": ["Затронутая метрика"]
        }}
    }}
]

БОЛИ ДЛЯ АНАЛИЗА:
{json.dumps(all_pains, ensure_ascii=False, indent=2)}'''

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _cross_analyze_interviews(self, summaries: List[InterviewSummary]) -> Dict[str, Any]:
        """Кросс-анализ всех интервью"""
        analysis_data = {
            'total_interviews': len(summaries),
            'profiles': [s.respondent_profile for s in summaries],
            'all_themes': [theme for s in summaries for theme in s.key_themes],
            'all_pains': [pain for s in summaries for pain in s.pain_points],
            'all_needs': [need for s in summaries for need in s.needs],
            'all_insights': [insight for s in summaries for insight in s.insights],
            'emotional_journeys': [ej for s in summaries for ej in s.emotional_journey],
            'brief_findings': [s.brief_related_findings for s in summaries if s.brief_related_findings]
        }

        context = self.brief_manager.get_brief_context()

        prompt = f'''{context}

Ты — ведущий аналитик с экспертизой в выявлении скрытых паттернов.

Проведи ГЛУБОЧАЙШИЙ кросс-анализ {len(summaries)} интервью с фокусом на достижение целей брифа.

КРИТИЧЕСКИ ВАЖНО:
1. Каждый паттерн подкрепляй МНОЖЕСТВОМ цитат
2. Связывай ВСЕ находки с целями и вопросами брифа
3. НЕ обобщай - приводи конкретные данные
4. Выявляй консенсус и расхождения

Верни детальный JSON:
{{
    "sample_characteristics": {{
        "total_respondents": {len(summaries)},
        "demographic_distribution": "Детальное описание на основе данных",
        "experience_distribution": "Распределение по опыту из интервью",
        "representativeness": "Оценка для целевой аудитории из брифа",
        "potential_biases": ["Возможное смещение с обоснованием"]
    }},
    "common_patterns": [
        {{
            "pattern": "Детальное описание паттерна из данных (мин. 80 слов)",
            "pattern_type": "behavioral/emotional/cognitive/social",
            "frequency": "6 из 8 респондентов (75%)",
            "confidence": "high/medium/low",
            "evidence": ["Конкретное доказательство 1", "Доказательство 2"],
            "quotes": ["ПОЛНАЯ цитата 1 минимум 60 слов", "ПОЛНАЯ цитата 2"],
            "exceptions": ["Исключение с обоснованием"],
            "underlying_need": "Глубинная потребность из данных",
            "design_implication": "Конкретная импликация",
            "business_implication": "Конкретная импликация",
            "relevance_to_brief": {{
                "goals": ["Релевантная цель"],
                "questions": ["Релевантный вопрос"],
                "metrics": ["Затронутая метрика"]
            }}
        }}
    ],
    "consensus_points": [
        {{
            "point": "Точка консенсуса из данных (50+ слов)",
            "agreement_level": "100% (8 из 8)",
            "quotes_sample": ["Цитата респондента 1", "Цитата респондента 3", "Цитата респондента 5"],
            "strength": "Сила убеждения на основе данных",
            "implication": "Конкретное значение для продукта",
            "action_required": "Что нужно сделать"
        }}
    ],
    "divergence_points": [
        {{
            "topic": "Тема расхождения из данных",
            "positions": [
                {{
                    "position": "Позиция 1 из интервью",
                    "holders": [1, 3, 5],
                    "reasoning": "Логика из их слов",
                    "quote": "Характерная цитата минимум 60 слов"
                }}
            ],
            "underlying_difference": "Что лежит в основе по данным",
            "segmentation_opportunity": "Конкретная возможность"
        }}
    ],
    "unexpected_connections": [
        {{
            "connection": "Неожиданная связь из данных",
            "evidence": ["Доказательство из интервью 1", "Из интервью 3"],
            "insight": "Инсайт основанный на связи",
            "hypothesis": "Гипотеза с обоснованием",
            "validation_needed": "Что нужно проверить"
        }}
    ],
    "brief_alignment": {{
        "goals_coverage": [
            {{
                "goal": "Цель из брифа",
                "coverage": "high/medium/low",
                "evidence_count": 15,
                "key_findings": ["Ключевая находка 1", "Находка 2"]
            }}
        ],
        "questions_answered": [
            {{
                "question": "Вопрос из брифа",
                "answer_quality": "comprehensive/partial/insufficient",
                "key_insights": ["Инсайт 1", "Инсайт 2"],
                "gaps": ["Что осталось неясным"]
            }}
        ]
    }},
    "meta_insights": [
        {{
            "insight": "Глубокий мета-инсайт из всех данных (80+ слов)",
            "based_on": "Конкретные данные и паттерны",
            "confidence": "high/medium/low",
            "paradigm_shift": "Какую парадигму меняет",
            "strategic_implication": "Стратегическое значение",
            "supporting_quotes": ["Цитата 1", "Цитата 2", "Цитата 3"]
        }}
    ]
}}

ДАННЫЕ ДЛЯ АНАЛИЗА:
Количество интервью: {len(summaries)}
Всего тем: {len(analysis_data['all_themes'])}
Всего болей: {len(analysis_data['all_pains'])}
Всего потребностей: {len(analysis_data['all_needs'])}

ДЕТАЛИ ИНТЕРВЬЮ:
{json.dumps([{
    'id': s.interview_id,
    'profile': s.respondent_profile,
    'themes': [t.get('theme') for t in s.key_themes][:3],
    'main_pains': [p.get('pain')[:100] + '...' for p in s.pain_points[:3]]
} for s in summaries], ensure_ascii=False, indent=2)}'''

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _identify_behavioral_patterns(self, summaries: List[InterviewSummary], cross_analysis: Dict) -> List[Dict]:
        """Выявление поведенческих паттернов"""
        context = self.brief_manager.get_brief_context()

        # Подготовка данных
        emotional_data = []
        for s in summaries:
            emotional_moments = []
            for e in s.emotional_journey[:5]:
                if isinstance(e, dict):
                    emotional_moments.append({
                        'moment': e.get('moment', ''),
                        'emotion': e.get('emotion', ''),
                        'trigger': e.get('trigger', ''),
                        'intensity': e.get('intensity', 0),
                        'quote': e.get('quote', '')
                    })

            emotional_data.append({
                'interview_id': s.interview_id,
                'emotional_moments': emotional_moments,
                'contradictions': s.contradictions[:3] if s.contradictions else []
            })

        prompt = f'''{context}

Ты — эксперт по поведенческому дизайну и человеческой психологии.

Выяви ВСЕ поведенческие паттерны из {len(summaries)} интервью.

КРИТИЧЕСКИ ВАЖНО:
1. Каждый паттерн основывай на РЕАЛЬНЫХ данных
2. Приводи МНОЖЕСТВО цитат для каждого паттерна
3. Связывай с целями исследования
4. НЕ придумывай - только из интервью

Верни JSON массив паттернов:
[
    {{
        "pattern_id": "BP001",
        "pattern": "Название паттерна из данных",
        "description": "Детальное описание на основе интервью (мин. 120 слов)",
        "pattern_type": "habit/ritual/workaround/avoidance/compensation/adaptive",
        "frequency": "75% респондентов (6 из 8)",
        "strength": "strong/moderate/weak",
        "evidence": [
            {{
                "type": "quote",
                "content": "ПОЛНАЯ цитата минимум 60 слов",
                "interview_ids": [1, 3, 5]
            }}
        ],
        "behavioral_sequence": [
            "Шаг 1: Точный триггер из интервью",
            "Шаг 2: Точное действие",
            "Шаг 3: Точный результат"
        ],
        "triggers": [
            {{
                "trigger": "Описание триггера из данных",
                "type": "situational/emotional/temporal",
                "reliability": "Надежность на основе данных",
                "quotes": ["Подтверждающая цитата"]
            }}
        ],
        "emotional_journey": [
            {{
                "stage": "До",
                "emotion": "Точная эмоция из интервью",
                "intensity": 7,
                "quote": "Подтверждающая цитата"
            }}
        ],
        "cost_to_user": {{
            "time": "Из интервью если есть",
            "cognitive_load": "На основе описаний",
            "emotional_toll": "Из слов респондентов",
            "opportunity_cost": "Что упускает по их словам"
        }},
        "business_implications": {{
            "impact": "На основе анализа",
            "metrics_affected": ["Метрика из брифа"],
            "revenue_impact": "Если можно оценить"
        }},
        "design_implications": {{
            "support_pattern": "Если позитивный",
            "break_pattern": "Если негативный",
            "intervention_points": ["Точка из анализа"]
        }},
        "relevance_to_brief": {{
            "helps_achieve_goals": ["Цель из брифа"],
            "answers_questions": ["Вопрос из брифа"],
            "affects_metrics": ["Метрика из брифа"]
        }},
        "representative_quotes": [
            "ПОЛНАЯ цитата 1 демонстрирующая паттерн (80+ слов)",
            "ПОЛНАЯ цитата 2 из другого интервью",
            "ПОЛНАЯ цитата 3"
        ]
    }}
]

ДАННЫЕ О ПАТТЕРНАХ:
{json.dumps(cross_analysis.get('common_patterns', []), ensure_ascii=False)}

ЭМОЦИОНАЛЬНЫЕ ДАННЫЕ:
{json.dumps(emotional_data, ensure_ascii=False)}'''

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _segment_audience(self, summaries: List[InterviewSummary], patterns: List[Dict]) -> List[Dict]:
        """Сегментация аудитории"""
        context = self.brief_manager.get_brief_context()

        prompt = f'''{context}

На основе {len(summaries)} интервью проведи ДЕТАЛЬНУЮ сегментацию аудитории.

КРИТИЧЕСКИ ВАЖНО:
1. Сегменты основывай ТОЛЬКО на реальных данных
2. Для каждого сегмента приводи респондентов
3. Связывай с целевой аудиторией из брифа
4. Подкрепляй характеристики цитатами

Создай 3-5 четких сегментов:

[
    {{
        "segment_id": "SEG001",
        "name": "Название основанное на данных",
        "description": "Детальное описание из интервью",
        "size": "30-40% (3-4 из 8-10 респондентов)",
        "demographics": {{
            "age_range": "Из данных интервью",
            "gender_distribution": "Из данных",
            "occupation_types": ["Из интервью"],
            "income_level": "Если упоминалось",
            "location": "Из данных"
        }},
        "psychographics": {{
            "values": ["Ценность из интервью"],
            "lifestyle": "Описание из данных",
            "motivations": ["Из слов респондентов"],
            "fears": ["Из интервью"]
        }},
        "behavioral_traits": {{
            "usage_patterns": ["Паттерн из данных"],
            "decision_making": "Из интервью",
            "technology_adoption": "Из поведения",
            "preferred_channels": ["Из упоминаний"]
        }},
        "pain_points": ["Боль характерная для сегмента"],
        "needs": ["Потребность сегмента"],
        "opportunities": ["Возможность для сегмента"],
        "interview_ids": [1, 3, 5],
        "representative_quotes": [
            "ПОЛНАЯ характерная цитата сегмента 1 (60+ слов)",
            "ПОЛНАЯ цитата 2",
            "ПОЛНАЯ цитата 3"
        ],
        "alignment_with_brief": {{
            "matches_target_audience": "yes/partially/no",
            "explanation": "Объяснение соответствия"
        }}
    }}
]

ДАННЫЕ РЕСПОНДЕНТОВ:
{json.dumps([{
    'id': s.interview_id,
    'profile': s.respondent_profile,
    'main_pains': [p.get('pain') for p in s.pain_points[:3]],
    'main_needs': [n.get('need') for n in s.needs[:3]]
} for s in summaries], ensure_ascii=False)}

ВЫЯВЛЕННЫЕ ПАТТЕРНЫ: {len(patterns)}'''

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _create_personas(self, segments: List[Dict], summaries: List[InterviewSummary]) -> List[Dict]:
        """Создание персон на основе реальных данных интервью"""
        context = self.brief_manager.get_brief_context()

        # Собираем реальные данные респондентов
        respondent_profiles = []
        for summary in summaries:
            # Извлекаем самые яркие цитаты
            top_quotes = []
            for quote in summary.quotes[:5]:  # Берем больше цитат
                if isinstance(quote, dict):
                    top_quotes.append(quote.get('text', ''))
                else:
                    top_quotes.append(str(quote))

            profile = {
                'id': summary.interview_id,
                'demographics': summary.respondent_profile.get('demographics', ''),
                'occupation': summary.respondent_profile.get('occupation', ''),
                'experience': summary.respondent_profile.get('experience_level', ''),
                'lifestyle': summary.respondent_profile.get('lifestyle', ''),
                'unique_traits': summary.respondent_profile.get('unique_traits', []),
                'key_quotes': top_quotes,
                'main_goals': [n.get('need', '') for n in summary.needs[:3] if n.get('need')],
                'main_pains': [p.get('pain', '') for p in summary.pain_points[:3] if p.get('pain')],
                'emotional_triggers': [e.get('trigger', '') for e in summary.emotional_journey[:2] if isinstance(e, dict)],
                'tech_literacy': summary.respondent_profile.get('tech_literacy', ''),
                'motivations': summary.respondent_profile.get('motivations', ''),
                'context': summary.respondent_profile.get('context', '')
            }
            respondent_profiles.append(profile)

        prompt = f'''{context}

Создай 3-4 УНИКАЛЬНЫЕ персоны на основе РЕАЛЬНЫХ данных респондентов.

КРИТИЧЕСКИ ВАЖНО:
1. Каждая персона = синтез 2-3 РЕАЛЬНЫХ респондентов
2. Используй ТОЛЬКО факты из интервью
3. НЕ придумывай детали - только из данных
4. Минимум 5 реальных цитат на персону
5. Связывай с целевой аудиторией брифа

[
    {{
        "persona_id": "P001",
        "name": "Имя отражающее характер (НЕ реальное имя)",
        "based_on_interviews": [1, 3, 5],
        "tagline": "РЕАЛЬНАЯ цитата характеризующая персону",
        "description": "Детальное описание ТОЛЬКО из данных респондентов",
        "demographics": {{
            "age": "Реальный возраст респондентов",
            "gender": "Из данных",
            "occupation": "Реальные профессии",
            "location": "Реальные локации",
            "family_status": "Если упоминалось",
            "income": "Если упоминалось",
            "education": "Из интервью"
        }},
        "real_life_context": {{
            "living_situation": "Из рассказов",
            "work_environment": "Реальный контекст",
            "daily_challenges": "Из интервью",
            "social_circle": "Если упоминалось",
            "typical_day": "Из описаний респондентов"
        }},
        "personality_traits": [
            "Черта выведенная из поведения",
            "Реальная характеристика"
        ],
        "goals": [
            "ТОЧНАЯ цель из интервью",
            "Конкретная потребность"
        ],
        "frustrations": [
            "ТОЧНАЯ фрустрация из данных",
            "Реальная боль"
        ],
        "needs": [
            "Специфическая потребность",
            "Реальная необходимость"
        ],
        "tech_behavior": {{
            "devices": "Из упоминаний",
            "apps_tools": "Реально используемые",
            "tech_comfort": "На основе поведения",
            "learning_style": "Из наблюдений"
        }},
        "real_quotes": [
            "ПОЛНАЯ ТОЧНАЯ цитата 1 (минимум 80 слов)",
            "ПОЛНАЯ цитата 2 из другого интервью той же персоны",
            "ПОЛНАЯ цитата 3",
            "Цитата 4",
            "Цитата 5"
        ],
        "typical_scenario": "РЕАЛЬНЫЙ сценарий из рассказов",
        "day_in_life": "На основе РЕАЛЬНЫХ историй",
        "decision_factors": [
            "Фактор из интервью",
            "Что действительно важно"
        ],
        "unique_details": [
            "Специфическая деталь 1",
            "Уникальная черта 2"
        ],
        "pain_point_quotes": [
            "Цитата о проблеме 1",
            "Цитата о проблеме 2"
        ],
        "solution_preferences": "Из слов респондентов",
        "alignment_with_target": {{
            "fits_brief_audience": "yes/partially/no",
            "explanation": "Почему да/нет"
        }}
    }}
]

РЕАЛЬНЫЕ ДАННЫЕ РЕСПОНДЕНТОВ:
{json.dumps(respondent_profiles, ensure_ascii=False)}

СЕГМЕНТЫ:
{json.dumps([{
    'name': s.get('name'),
    'size': s.get('size'),
    'interview_ids': s.get('interview_ids', [])
} for s in segments], ensure_ascii=False)}

ИСПОЛЬЗУЙ ТОЛЬКО ЭТИ ДАННЫЕ! НЕ ВЫДУМЫВАЙ!'''

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _generate_final_findings(self, summaries: List[InterviewSummary],
                              cross_analysis: Dict, patterns: List[Dict],
                              segments: List[Dict], personas: List[Dict]) -> ResearchFindings:
        """Генерация финальных выводов"""
        context = self.brief_manager.get_brief_context()

        # Собираем ключевые данные
        all_pains = []
        for summary in summaries:
            for p in summary.pain_points:
                all_pains.append({
                    'pain': p.get('pain', ''),
                    'severity': p.get('severity', 'medium'),
                    'quotes': p.get('quotes', []),
                    'relevance_to_brief': p.get('relevance_to_brief', '')
                })

        prompt = f'''{context}

Ты — стратегический директор по продуктам. Синтезируй ВСЕ данные в actionable выводы для C-level.

КРИТИЧЕСКИ ВАЖНО:
1. Каждый вывод подкрепляй МНОЖЕСТВОМ цитат
2. Показывай достижение КАЖДОЙ цели брифа
3. Отвечай на КАЖДЫЙ вопрос брифа
4. НЕ обобщай - давай конкретику с числами

Верни детальный JSON:
{{
    "executive_summary": "Исчерпывающее резюме (300-400 слов). Начни с достижения главной цели брифа, затем ключевые находки по каждому вопросу с точными числами, закончи критическими действиями.",

    "key_insights": [
        {{
            "insight_id": "KI001",
            "problem_title": "Краткое название проблемы",
            "problem_statement": "Когда [точная ситуация], пользователи [точная проблема], что приводит к [точное последствие]",
            "problem_description": "Исчерпывающее описание на основе данных (мин. 200 слов)",
            "severity": "critical/high/medium",
            "affected_percentage": "75% (6 из 8)",
            "business_impact": {{
                "metric": "Конкретная метрика",
                "current_impact": "Точные текущие потери",
                "potential_impact": "Точный потенциал роста"
            }},
            "root_cause": "Глубинная причина из анализа",
            "evidence": ["Конкретное доказательство 1", "Доказательство 2", "Доказательство 3"],
            "quotes": [
                {{
                    "text": "ПОЛНАЯ цитата (80+ слов)",
                    "interview_id": 1,
                    "context": "Контекст цитаты"
                }}
            ],
            "opportunity": {{
                "description": "Детальная возможность из данных (120+ слов)",
                "value_prop": "Конкретное ценностное предложение",
                "implementation": "Конкретный подход"
            }},
            "relevance_to_brief": {{
                "addresses_goal": "Какую цель помогает достичь",
                "answers_question": "На какой вопрос отвечает",
                "impacts_metric": "На какую метрику влияет"
            }},
            "priority": "P0/P1/P2",
            "effort": "S/M/L/XL"
        }}
    ],

    "brief_achievement": {{
        "goals_status": [
            {{
                "goal": "Цель из брифа",
                "achievement_level": "fully/partially/not achieved",
                "evidence": ["Доказательство 1", "Доказательство 2"],
                "key_findings": ["Находка 1", "Находка 2"],
                "gaps": ["Что не удалось если есть"]
            }}
        ],
        "questions_answers": [
            {{
                "question": "Вопрос из брифа",
                "answer": "Полный ответ на основе данных (100+ слов)",
                "confidence": "high/medium/low",
                "supporting_data": ["Данные 1", "Данные 2"],
                "quotes": ["Подтверждающая цитата 1", "Цитата 2"]
            }}
        ],
        "metrics_impact": [
            {{
                "metric": "Метрика из брифа",
                "current_state": "Текущее состояние",
                "projected_improvement": "Прогноз улучшения",
                "required_actions": ["Действие 1", "Действие 2"]
            }}
        ]
    }},

    "paradigm_shifts": [
        {{
            "from": "Текущий подход из данных",
            "to": "Необходимый подход из анализа",
            "why": "Обоснование на данных",
            "evidence": ["Доказательство из интервью"],
            "implementation": "Конкретный путь",
            "expected_results": "Конкретные результаты"
        }}
    ],

    "strategic_recommendations": [
        {{
            "recommendation": "Конкретная рекомендация",
            "rationale": "Детальное обоснование на данных",
            "expected_outcome": "Конкретный результат",
            "timeline": "Точные сроки",
            "investment": "Конкретная оценка",
            "risks": ["Риск 1", "Риск 2"],
            "success_metrics": ["Метрика 1", "Метрика 2"]
        }}
    ],

    "critical_quotes": [
        "Самая важная цитата 1 (100+ слов) - Интервью X",
        "Критическая цитата 2 - Интервью Y",
        "Ключевая цитата 3 - Интервью Z"
    ],

    "next_research": [
        "Конкретный вопрос для изучения 1",
        "Вопрос 2 с обоснованием"
    ]
}}

ДАННЫЕ АНАЛИЗА:
- Интервью: {len(summaries)}
- Паттернов: {len(patterns)}
- Сегментов: {len(segments)}
- Персон: {len(personas)}

КЛЮЧЕВЫЕ БОЛИ (все):
{json.dumps(all_pains, ensure_ascii=False)}

ДОСТИЖЕНИЕ ЦЕЛЕЙ:
{json.dumps(cross_analysis.get('brief_alignment', {}), ensure_ascii=False)}'''

        response = self.api_wrapper.generate_content(prompt)
        findings_data = self._extract_json(response)

        return ResearchFindings(
            executive_summary=findings_data.get('executive_summary', ''),
            key_insights=findings_data.get('key_insights', []),
            behavioral_patterns=patterns,
            user_segments=segments,
            pain_points_map={},
            opportunities=findings_data.get('strategic_recommendations', []),
            recommendations=findings_data.get('strategic_recommendations', []),
            risks=findings_data.get('critical_assumptions', []),
            personas=personas,
            brief_answers=findings_data.get('brief_achievement', {}),
            goal_achievement=findings_data.get('brief_achievement', {})
        )

    def _generate_current_metrics(self, summaries: List[InterviewSummary]) -> Dict[str, Any]:
        """Генерация текущих метрик на основе реальных данных"""
        if not summaries:
            return {
                'estimated_nps': 'Недостаточно данных',
                'churn_risk': 'Не определен',
                'avg_pains_per_user': 0,
                'avg_needs_per_user': 0,
                'negative_emotion_ratio': 0,
                'satisfaction_score': 'Не определен'
            }

        total_pains = sum(len(s.pain_points) for s in summaries)
        total_needs = sum(len(s.needs) for s in summaries)

        # Анализ эмоций
        negative_emotions = 0
        positive_emotions = 0
        neutral_emotions = 0

        negative_keywords = ['негатив', 'раздражение', 'фрустрация', 'злость', 'разочарование',
                           'недовольство', 'гнев', 'страх', 'тревога', 'беспокойство']
        positive_keywords = ['радость', 'удовлетворение', 'восторг', 'счастье', 'довольство',
                           'удовольствие', 'энтузиазм', 'воодушевление']

        for summary in summaries:
            for journey in summary.emotional_journey:
                if isinstance(journey, dict):
                    emotion = journey.get('emotion', '').lower()
                    if any(neg in emotion for neg in negative_keywords):
                        negative_emotions += 1
                    elif any(pos in emotion for pos in positive_keywords):
                        positive_emotions += 1
                    else:
                        neutral_emotions += 1

        total_emotions = negative_emotions + positive_emotions + neutral_emotions

        # NPS на основе реальных данных
        if total_emotions > 0:
            positive_ratio = positive_emotions / total_emotions
            negative_ratio = negative_emotions / total_emotions
            estimated_nps = int((positive_ratio - negative_ratio) * 100)
        else:
            # Используем sentiment анализ
            sentiments = [s.sentiment_score for s in summaries if s.sentiment_score != 0]
            if sentiments:
                avg_sentiment = np.mean(sentiments)
                estimated_nps = int(avg_sentiment * 100)
            else:
                estimated_nps = 'Недостаточно данных'

        # Риск оттока на основе данных
        avg_pains = total_pains / len(summaries) if summaries else 0

        if isinstance(estimated_nps, int):
            if estimated_nps < -30 or avg_pains > 7:
                churn_risk = 'Критический'
            elif estimated_nps < 0 or avg_pains > 5:
                churn_risk = 'Высокий'
            elif estimated_nps < 30 or avg_pains > 3:
                churn_risk = 'Средний'
            else:
                churn_risk = 'Низкий'
        else:
            if avg_pains > 7:
                churn_risk = 'Критический'
            elif avg_pains > 4:
                churn_risk = 'Высокий'
            else:
                churn_risk = 'Средний'

        # Удовлетворенность
        if total_emotions > 0:
            satisfaction_score = round((positive_emotions / total_emotions * 4) + 1, 1)
        else:
            satisfaction_score = 'Недостаточно данных'

        return {
            'estimated_nps': estimated_nps,
            'churn_risk': churn_risk,
            'avg_pains_per_user': round(avg_pains, 1),
            'avg_needs_per_user': round(total_needs / len(summaries), 1) if summaries else 0,
            'negative_emotion_ratio': round(negative_emotions / total_emotions * 100 if total_emotions > 0 else 0),
            'satisfaction_score': satisfaction_score,
            'total_emotions_analyzed': total_emotions,
            'sample_size': len(summaries)
        }

    def _format_insights_for_report(self, insights: List[Dict]) -> List[Dict]:
        """Форматирование инсайтов для отчета"""
        formatted_insights = []

        for insight in insights:
            formatted_insights.append({
                "title": insight.get("problem_title", ""),
                "description": insight.get("problem_description", ""),
                "opportunity": insight.get("opportunity", {}).get("description", "") if isinstance(insight.get("opportunity"), dict) else insight.get("opportunity", ""),
                "evidence": insight.get("evidence", []),
                "severity": insight.get("severity", "medium"),
                "priority": insight.get("priority", "P2"),
                "quotes": insight.get("quotes", []),
                "relevance_to_brief": insight.get("relevance_to_brief", {})
            })

        return formatted_insights

    @retry_on_overload
    def _generate_recommendations(self, insights: List[Dict]) -> Dict:
        """Генерация рекомендаций на основе реальных данных"""
        context = self.brief_manager.get_brief_context()

        prompt = f'''{context}

На основе выявленных проблем создай МАКСИМАЛЬНО КОНКРЕТНЫЕ рекомендации.

КРИТИЧЕСКИ ВАЖНО:
- Каждая рекомендация решает РЕАЛЬНУЮ проблему из данных
- Все предложения основаны на том, что сказали респонденты
- Метрики успеха связаны с целями брифа
- НЕ придумывай решения - выводи из данных

Верни детальный JSON:
{{
    "quick_wins": [
        {{
            "title": "Конкретное решение",
            "description": "Что именно сделать на основе данных",
            "based_on_insights": ["Инсайт на котором основано"],
            "user_quotes_supporting": ["Цитата поддерживающая решение"],
            "implementation_steps": [
                "Конкретный шаг 1",
                "Конкретный шаг 2",
                "Конкретный шаг 3"
            ],
            "expected_impact": "Конкретный эффект с обоснованием",
            "affected_problems": ["Проблема которую решает"],
            "timeline": "Реалистичный срок",
            "resources_needed": "Конкретные ресурсы",
            "success_metrics": ["Метрика из брифа"],
            "risks": ["Риск на основе данных"],
            "dependencies": ["Зависимость"],
            "user_validation": "Как проверить с пользователями"
        }}
    ],
    "strategic_initiatives": [
        {{
            "title": "Стратегическая инициатива из анализа",
            "description": "Детальное описание",
            "rationale": "Обоснование на данных исследования",
            "supporting_patterns": ["Паттерн который поддерживает"],
            "expected_roi": "ROI на основе данных",
            "implementation_phases": ["Фаза 1", "Фаза 2"],
            "success_criteria": ["Критерий из данных"],
            "alignment_with_brief": "Как достигает целей брифа"
        }}
    ],
    "innovation_opportunities": [
        {{
            "title": "Инновация вытекающая из инсайтов",
            "description": "Описание на основе находок",
            "based_on": ["Неожиданная связь из данных"],
            "potential_impact": "Влияние с обоснованием",
            "required_research": "Что нужно изучить дополнительно",
            "user_readiness": "Готовность пользователей из данных"
        }}
    ]
}}

КЛЮЧЕВЫЕ ПРОБЛЕМЫ ДЛЯ РЕШЕНИЯ:
{json.dumps(insights, ensure_ascii=False, indent=2)}'''

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _generate_defense_materials(self, findings: ResearchFindings, recommendations: Dict, total_interviews: int) -> Dict:
        """Генерация материалов для защиты на основе реальных данных"""
        context = self.brief_manager.get_brief_context()

        # Собираем реальную статистику из findings
        key_findings = []

        # Генерируем ключевые выводы на основе реальных данных
        if findings.key_insights:
            for i, insight in enumerate(findings.key_insights[:5], 1):
                affected_percentage = insight.get('affected_percentage', '')
                problem_title = insight.get('problem_title', 'Проблема')
                quotes = insight.get('quotes', [])

                # Формируем вывод на основе реальных данных
                if affected_percentage:
                    finding = f"{affected_percentage} респондентов испытывают проблему: {problem_title}"
                else:
                    finding = f"Выявлена проблема: {problem_title}"

                # Добавляем цитату если есть
                if quotes and isinstance(quotes[0], dict):
                    quote_text = quotes[0].get('text', '')
                    if quote_text:
                        finding += f' - "{quote_text[:100]}..."'

                key_findings.append(finding)

        # Добавляем выводы на основе реальных метрик
        if findings.current_metrics:
            metrics = findings.current_metrics
            if metrics.get('avg_pains_per_user', 0) > 0:
                key_findings.append(f"В среднем {metrics['avg_pains_per_user']} проблем на пользователя (на основе {total_interviews} интервью)")

            if isinstance(metrics.get('estimated_nps'), int):
                key_findings.append(f"Предполагаемый NPS: {metrics['estimated_nps']} (на основе эмоционального анализа)")

        problems_summary = json.dumps(findings.key_insights[:5], ensure_ascii=False)
        recs_summary = json.dumps(recommendations.get("quick_wins", [])[:5], ensure_ascii=False)

        prompt = f'''{context}

Создай материалы для защиты отчета перед заказчиком на основе РЕАЛЬНЫХ данных из {total_interviews} интервью.

ВАЖНО:
- Все материалы должны показывать достижение целей брифа
- Используй ТОЛЬКО реальную статистику и данные
- НЕ придумывай числа - только из анализа

Верни JSON:
{{
    "executive_summary": "Резюме для руководства (7-10 предложений) на основе реальных данных из {total_interviews} интервью. Укажи точные числа и проценты из анализа.",

    "key_findings": {json.dumps(key_findings, ensure_ascii=False) if key_findings else '["Ключевая находка 1 из анализа", "Находка 2", "Находка 3", "Находка 4", "Находка 5"]'},

    "brief_achievement_summary": {{
        "goals_achieved": ["Цель 1: достигнута через...", "Цель 2: частично достигнута..."],
        "questions_answered": ["Вопрос 1: ответ...", "Вопрос 2: ответ..."],
        "metrics_impact": ["Метрика 1: прогноз улучшения...", "Метрика 2: влияние..."]
    }},

    "speaker_notes": [
        "Слайд 1: Начните с главного достижения - провели анализ {total_interviews} интервью",
        "Слайд 2: Покажите самую критичную проблему с реальной цитатой",
        "Слайд 3: Представьте ключевые числа из анализа",
        "Слайд 4: Ответьте на главный вопрос брифа на основе данных",
        "Слайд 5: Покажите quick win с обоснованием из интервью",
        "Слайд 6: Адресуйте возможные сомнения данными",
        "Слайд 7: Завершите конкретными шагами"
    ],

    "potential_questions": [
        {{
            "question": "Насколько репрезентативна выборка?",
            "answer": "Проанализировано {total_interviews} интервью. {'Это меньше рекомендуемых 8, но данные показывают четкие паттерны.' if total_interviews < 8 else 'Выборка достаточна для выявления основных паттернов.'}"
        }},
        {{
            "question": "Как вы пришли к этим выводам?",
            "answer": "Использован AI-ассистированный анализ с фокусом на точные цитаты (минимум 50 слов). Каждый вывод подкреплен множественными доказательствами."
        }},
        {{
            "question": "Что если мы ничего не будем делать?",
            "answer": "На основе данных: текущий риск оттока - [из метрик], среднее количество проблем - [из метрик]. Без действий ситуация ухудшится."
        }},
        {{
            "question": "Почему именно эти рекомендации?",
            "answer": "Каждая рекомендация основана на конкретных проблемах из интервью и поддержана цитатами пользователей."
        }},
        {{
            "question": "Как измерить успех?",
            "answer": "Через метрики из брифа + снижение количества проблем на пользователя + улучшение NPS"
        }}
    ],

    "success_metrics": [
        "Количество интервью: {total_interviews}",
        "Выявлено проблем: [реальное число из анализа]",
        "Средние боли на пользователя: [из метрик]",
        "[Метрика из брифа]: прогноз на основе данных"
    ],

    "next_steps": [
        {{
            "action": "Валидация quick wins с пользователями",
            "timeline": "1-2 недели",
            "responsible": "Продуктовая команда",
            "resources": "5-7 пользователей для тестов",
            "expected_result": "Подтверждение гипотез"
        }},
        {{
            "action": "Запуск пилота решения #1",
            "timeline": "3-4 недели",
            "responsible": "Dev team + дизайн",
            "resources": "2 разработчика, 1 дизайнер",
            "expected_result": "MVP для 10% пользователей"
        }},
        {{
            "action": "Мониторинг метрик",
            "timeline": "Постоянно",
            "responsible": "Аналитика",
            "resources": "Дашборд + алерты",
            "expected_result": "Еженедельные отчеты"
        }}
    ],

    "roi_calculation": "На основе {total_interviews} интервью: [конкретные расчеты из данных]",

    "risk_mitigation": [
        {{
            "risk": "Малая выборка" if total_interviews < 8 else "Сопротивление изменениям",
            "mitigation": "Дополнительная валидация" if total_interviews < 8 else "Поэтапное внедрение с A/B тестами",
            "monitoring": "Расширение исследования" if total_interviews < 8 else "Еженедельные отчеты по adoption"
        }},
        {{
            "risk": "Технические ограничения",
            "mitigation": "Начать с простых решений",
            "monitoring": "Техническая оценка перед стартом"
        }}
    ],

    "powerful_quotes": [
        "Самая убедительная цитата из интервью о главной проблеме",
        "Цитата показывающая упущенную выгоду",
        "Эмоциональная цитата из данных"
    ]
}}

ОСНОВНЫЕ НАХОДКИ:
{problems_summary}

РЕКОМЕНДАЦИИ:
{recs_summary}

РЕАЛЬНЫЕ МЕТРИКИ:
{json.dumps(findings.current_metrics, ensure_ascii=False)}

ВСЕГО ИНТЕРВЬЮ: {total_interviews}'''

        response = self.api_wrapper.generate_content(prompt)
        result = self._extract_json(response)

        # Убеждаемся, что key_findings содержит реальные данные
        if not result.get('key_findings') or result['key_findings'] == ["Ключевая находка 1 из анализа", "Находка 2", "Находка 3", "Находка 4", "Находка 5"]:
            result['key_findings'] = key_findings

        return result

    @retry_on_overload
    def _analyze_brief_questions(self, summaries: List[InterviewSummary], findings: ResearchFindings) -> Dict:
        """Анализ ответов на вопросы брифа"""
        if not self.brief_manager.has_brief:
            return {}

        context = self.brief_manager.get_brief_context()
        questions = self.brief_manager.get_questions_for_analysis()

        # Собираем все данные связанные с вопросами
        all_brief_findings = []
        for summary in summaries:
            if summary.brief_related_findings:
                all_brief_findings.append({
                    'interview_id': summary.interview_id,
                    'findings': summary.brief_related_findings
                })

        prompt = f'''{context}

Проанализируй ВСЕ данные и дай ИСЧЕРПЫВАЮЩИЕ ответы на вопросы исследования.

ВОПРОСЫ ИССЛЕДОВАНИЯ:
{json.dumps(questions, ensure_ascii=False)}

Для КАЖДОГО вопроса:
1. Синтезируй ВСЕ релевантные данные
2. Дай четкий, полный ответ
3. Подкрепи МНОЖЕСТВОМ цитат
4. Укажи уверенность в ответе

Верни JSON:
{{
    "answers": [
        {{
            "question": "Точный вопрос из брифа",
            "answer": "ПОЛНЫЙ детальный ответ на основе всех данных (минимум 150 слов)",
            "confidence": "high/medium/low",
            "answer_summary": "Краткий ответ в 1-2 предложения",
            "supporting_evidence": [
                {{
                    "type": "pattern/quote/statistic",
                    "content": "Конкретное доказательство",
                    "source": "Интервью X / Паттерн Y"
                }}
            ],
            "key_quotes": [
                {{
                    "quote": "ПОЛНАЯ цитата подтверждающая ответ (80+ слов)",
                    "interview_id": 1,
                    "relevance": "Как цитата отвечает на вопрос"
                }}
            ],
            "data_gaps": ["Что осталось неясным"],
            "recommendations": ["Что делать на основе ответа"]
        }}
    ],
    "cross_question_insights": [
        {{
            "insight": "Инсайт связывающий несколько вопросов",
            "related_questions": ["Вопрос 1", "Вопрос 2"],
            "implication": "Что это значит для продукта"
        }}
    ],
    "unexpected_findings": [
        {{
            "finding": "Находка не связанная напрямую с вопросами",
            "importance": "Почему это важно",
            "recommendation": "Что с этим делать"
        }}
    ]
}}

ДАННЫЕ ИЗ ИНТЕРВЬЮ:
{json.dumps(all_brief_findings, ensure_ascii=False)}

КЛЮЧЕВЫЕ ИНСАЙТЫ:
{json.dumps([{
    'title': i.get('problem_title'),
    'description': i.get('problem_description'),
    'relevance': i.get('relevance_to_brief')
} for i in findings.key_insights[:5]], ensure_ascii=False)}'''

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    @retry_on_overload
    def _assess_goal_achievement(self, findings: ResearchFindings, summaries: List[InterviewSummary]) -> Dict:
        """Оценка достижения целей исследования"""
        if not self.brief_manager.has_brief:
            return {}

        context = self.brief_manager.get_brief_context()
        goals = self.brief_manager.get_goals_for_analysis()

        prompt = f'''{context}

Оцени достижение КАЖДОЙ цели исследования на основе всех данных.

ЦЕЛИ ИССЛЕДОВАНИЯ:
{json.dumps(goals, ensure_ascii=False)}

Для КАЖДОЙ цели определи:
1. Уровень достижения
2. Конкретные результаты
3. Доказательства достижения
4. Что не удалось и почему

Верни JSON:
{{
    "goals": [
        {{
            "goal": "Точная цель из брифа",
            "achievement_level": "fully_achieved/mostly_achieved/partially_achieved/not_achieved",
            "achievement_percentage": 85,
            "results": [
                "Конкретный результат 1 с числами",
                "Результат 2 с данными",
                "Результат 3"
            ],
            "evidence": [
                {{
                    "type": "insight/pattern/quote",
                    "content": "Конкретное доказательство",
                    "strength": "strong/moderate/weak"
                }}
            ],
            "key_findings": [
                "Ключевая находка относящаяся к цели",
                "Еще находка"
            ],
            "gaps": ["Что не удалось выяснить"],
            "recommendations": ["Рекомендация для полного достижения"],
            "quotes": [
                "Цитата подтверждающая достижение цели"
            ]
        }}
    ],
    "overall_success": {{
        "success_rate": "75% (3 из 4 целей достигнуты)",
        "key_achievements": [
            "Главное достижение 1",
            "Достижение 2"
        ],
        "main_gaps": [
            "Главный пробел 1"
        ],
        "next_steps": [
            "Следующий шаг для закрытия пробелов"
        ]
    }},
    "metrics_progress": [
        {{
            "metric": "Метрика из брифа",
            "baseline": "Текущее значение из данных",
            "target": "Целевое значение",
            "projected_improvement": "Прогноз улучшения",
            "confidence": "high/medium/low",
            "required_actions": ["Действие 1", "Действие 2"]
        }}
    ]
}}

КОЛИЧЕСТВО ИНТЕРВЬЮ: {len(summaries)}
КЛЮЧЕВЫХ ИНСАЙТОВ: {len(findings.key_insights)}
ВЫЯВЛЕННЫХ ПАТТЕРНОВ: {len(findings.behavioral_patterns)}
СЕГМЕНТОВ: {len(findings.user_segments)}'''

        response = self.api_wrapper.generate_content(prompt)
        return self._extract_json(response)

    def _extract_json(self, text: str) -> Union[Dict, List]:
        """Извлечение JSON из ответа Gemini"""
        try:
            # Попытка 1: весь ответ - JSON
            try:
                return json.loads(text)
            except:
                pass

            # Попытка 2: JSON между ```json и ```
            json_match = re.search(r'```json\s*(.*?)\s*```', text, re.DOTALL)
            if json_match:
                return json.loads(json_match.group(1))

            # Попытка 3: JSON между { и }
            json_match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', text, re.DOTALL)
            if json_match:
                return json.loads(json_match.group(0))

            # Попытка 4: JSON массив между [ и ]
            json_match = re.search(r'\[[^\[\]]*(?:\[[^\[\]]*\][^\[\]]*)*\]', text, re.DOTALL)
            if json_match:
                return json.loads(json_match.group(0))

            logging.warning(f"Не удалось извлечь JSON из ответа. Первые 500 символов: {text[:500]}")
            return {}

        except Exception as e:
            logging.error(f"Ошибка при извлечении JSON: {e}")
            return {}

# ========================================================================
# ГЕНЕРАТОР ОТЧЕТОВ
# ========================================================================
class EnhancedReportGeneratorFixed:
    def __init__(self, company_config):
        self.config = company_config
        self.colors = {
            'primary': '#2563eb',
            'secondary': '#7c3aed',
            'success': '#10b981',
            'warning': '#f59e0b',
            'danger': '#ef4444',
            'dark': '#1f2937',
            'light': '#f3f4f6',
            'white': '#ffffff',
            'text': '#111827',
            'text_secondary': '#6b7280',
            'border': '#e5e7eb',
            'background': '#f9fafb'
        }

    def generate_html(self, analysis_data):
        """Генерация HTML отчета с учетом брифа"""
        # Извлекаем данные брифа
        brief_data = analysis_data.get('brief_data', None)
        brief_answers = analysis_data.get('brief_answers', {})
        goal_achievement = analysis_data.get('goal_achievement', {})

        # Подготовка данных
        base = analysis_data.get('base_analysis', {})
        recs = analysis_data.get('recommendations', {})
        defense = analysis_data.get('defense_materials', {})
        summaries = analysis_data.get('interview_summaries', [])
        findings = analysis_data.get('findings', {})
        total_interviews = analysis_data.get('total_interviews', len(summaries))
        current_metrics = analysis_data.get('current_metrics', {})
        personas = analysis_data.get('personas', [])

        # Генерация статичных графиков
        charts = self._generate_static_charts(analysis_data)

        # Проверяем наличие данных
        has_brief = brief_data is not None
        has_segments = len(base.get('segments', [])) > 0
        has_personas = len(personas) > 0
        has_problems = len(base.get('problems', [])) > 0
        has_patterns = len(findings.behavioral_patterns if findings else []) > 0
        has_insights = len(base.get('insights', [])) > 0
        has_recommendations = len(recs.get('quick_wins', [])) > 0
        has_brief_answers = bool(brief_answers.get('answers', []))
        has_goal_achievement = bool(goal_achievement.get('goals', []))

        # HTML со всеми разделами
        html = f'''<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{self.config.report_title}</title>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800;900&display=swap" rel="stylesheet">
    <style>
        {self._get_professional_css()}
    </style>
</head>
<body>
    {self._generate_cover_page()}
    {self._generate_table_of_contents(analysis_data)}
    {self._generate_brief_section(brief_data) if has_brief else ''}
    {self._generate_overview_section(analysis_data, charts)}
    {self._generate_goal_achievement_section(goal_achievement) if has_goal_achievement else ''}
    {self._generate_brief_answers_section(brief_answers) if has_brief_answers else ''}
    {self._generate_current_state_section(current_metrics, charts) if current_metrics.get('estimated_nps') != 'Недостаточно данных' else ''}
    {self._generate_key_metrics_section(analysis_data, charts) if defense.get('key_findings', []) else ''}
    {self._generate_user_segments_section(base.get('segments', []), charts) if has_segments else ''}
    {self._generate_personas_section(personas) if has_personas else ''}
    {self._generate_pain_points_section_full(base.get('problems', []), charts) if has_problems else ''}
    {self._generate_behavioral_patterns_section_full(findings.behavioral_patterns if findings else []) if has_patterns else ''}
    {self._generate_emotional_journey_section(summaries)}
    {self._generate_insights_section_full(base.get('insights', [])) if has_insights else ''}
    {self._generate_contradictions_section(summaries)}
    {self._generate_quotes_section(summaries)}
    {self._generate_recommendations_section_full(recs) if has_recommendations else ''}
    {self._generate_priority_matrix_section(recs) if has_recommendations else ''}
    {self._generate_roadmap_section(defense) if defense.get('next_steps', []) else ''}
    {self._generate_detailed_appendix(summaries, analysis_data)}
</body>
</html>'''

        return html

    def _generate_brief_section(self, brief_data):
        """Генерация раздела с брифом исследования"""
        if not brief_data:
            return ''

        goals_html = ''
        if brief_data.get('research_goals'):
            goals_html = '<h3>Цели исследования</h3><ul>'
            for goal in brief_data['research_goals']:
                goals_html += f'<li>{goal}</li>'
            goals_html += '</ul>'

        questions_html = ''
        if brief_data.get('research_questions'):
            questions_html = '<h3>Исследовательские вопросы</h3><ul>'
            for question in brief_data['research_questions']:
                questions_html += f'<li>{question}</li>'
            questions_html += '</ul>'

        metrics_html = ''
        if brief_data.get('success_metrics'):
            metrics_html = '<h3>Метрики успеха</h3><ul>'
            for metric in brief_data['success_metrics']:
                metrics_html += f'<li>{metric}</li>'
            metrics_html += '</ul>'

        # Исправляем проблемные части
        audience_html = ''
        if brief_data.get('target_audience'):
            audience_html = f'<h3>Целевая аудитория</h3><p>{brief_data.get("target_audience", "")}</p>'

        context_html = ''
        if brief_data.get('business_context'):
            context_html = f'<h3>Бизнес-контекст</h3><p>{brief_data.get("business_context", "")}</p>'

        constraints_html = ''
        if brief_data.get('constraints'):
            constraints_list = ''.join([f'<li>{c}</li>' for c in brief_data.get('constraints', [])])
            constraints_html = f'<h3>Ограничения</h3><ul>{constraints_list}</ul>'

        return f'''
        <div class="page" id="brief">
            <div class="container">
                <h2>Контекст исследования</h2>

                {goals_html}
                {questions_html}
                {audience_html}
                {context_html}
                {metrics_html}
                {constraints_html}
            </div>
        </div>
        '''

    def _generate_goal_achievement_section(self, goal_achievement):
        """Генерация раздела достижения целей"""
        if not goal_achievement or not goal_achievement.get('goals'):
            return ''

        goals_html = ''
        for goal_data in goal_achievement.get('goals', []):
            achievement_level = goal_data.get('achievement_level', 'not_achieved')
            achievement_percentage = goal_data.get('achievement_percentage', 0)

            # Цвет в зависимости от уровня достижения
            if achievement_level == 'fully_achieved':
                color = self.colors['success']
                status_text = 'Полностью достигнута'
            elif achievement_level == 'mostly_achieved':
                color = self.colors['primary']
                status_text = 'В основном достигнута'
            elif achievement_level == 'partially_achieved':
                color = self.colors['warning']
                status_text = 'Частично достигнута'
            else:
                color = self.colors['danger']
                status_text = 'Не достигнута'

            # Результаты
            results_html = ''
            if goal_data.get('results'):
                results_html = '<h4>Результаты:</h4><ul>'
                for result in goal_data['results']:
                    results_html += f'<li>{result}</li>'
                results_html += '</ul>'

            # Доказательства
            evidence_html = ''
            if goal_data.get('evidence'):
                evidence_html = '<h4>Доказательства:</h4><div style="margin-bottom: 20px;">'
                for e in goal_data['evidence']:
                    if isinstance(e, dict):
                        strength = e.get('strength', 'moderate')
                        strength_color = self.colors['success'] if strength == 'strong' else self.colors['warning'] if strength == 'moderate' else self.colors['danger']
                        evidence_html += f'''
                        <div class="card" style="margin: 10px 0; border-left: 3px solid {strength_color};">
                            <p>{e.get('content', '')}</p>
                            <span class="tag">Сила: {strength}</span>
                        </div>
                        '''
                evidence_html += '</div>'

            # Цитаты
            quotes_html = ''
            if goal_data.get('quotes'):
                quotes_html = '<h4>Подтверждающие цитаты:</h4>'
                for quote in goal_data['quotes']:
                    quotes_html += f'''
                    <div class="quote-card" style="margin: 10px 0;">
                        <p class="quote-text">{quote}</p>
                    </div>
                    '''

            # Пробелы
            gaps_html = ''
            if goal_data.get('gaps'):
                gaps_html = '<h4 style="color: ' + self.colors['danger'] + ';">Что не удалось выяснить:</h4><ul>'
                for gap in goal_data['gaps']:
                    gaps_html += f'<li>{gap}</li>'
                gaps_html += '</ul>'

            # Рекомендации
            recommendations_html = ''
            if goal_data.get('recommendations'):
                recs_list = ''.join([f'<li>{r}</li>' for r in goal_data.get('recommendations', [])])
                recommendations_html = f'<div style="margin-top: 20px; padding: 15px; background: {self.colors["background"]}; border-radius: 8px;"><h4>Рекомендации:</h4><ul>{recs_list}</ul></div>'

            goals_html += f'''
            <div class="card" style="margin-bottom: 30px; border-top: 4px solid {color};">
                <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 20px;">
                    <h3>Цель: {goal_data.get('goal', '')}</h3>
                    <div>
                        <span class="tag" style="background: {color}; color: white;">{status_text}</span>
                        <span class="metric-value" style="color: {color}; margin-left: 20px;">{achievement_percentage}%</span>
                    </div>
                </div>

                {results_html}
                {evidence_html}
                {quotes_html}
                {gaps_html}
                {recommendations_html}
            </div>
            '''

        # Общий прогресс
        overall = goal_achievement.get('overall_success', {})

        # Исправляем проблемные f-строки
        achievements_html = ''
        if overall.get('key_achievements'):
            achievements_list = ''.join([f'<li>{a}</li>' for a in overall.get('key_achievements', [])])
            achievements_html = f'<h4>Ключевые достижения:</h4><ul>{achievements_list}</ul>'

        gaps_html = ''
        if overall.get('main_gaps'):
            gaps_list = ''.join([f'<li>{g}</li>' for g in overall.get('main_gaps', [])])
            gaps_html = f'<h4 style="color: {self.colors["warning"]};">Основные пробелы:</h4><ul>{gaps_list}</ul>'

        return f'''
        <div class="page" id="goal-achievement">
            <div class="container">
                <h2>Достижение целей исследования</h2>

                <div class="card" style="background: linear-gradient(135deg, {self.colors['background']} 0%, rgba(37, 99, 235, 0.05) 100%); margin-bottom: 30px;">
                    <h3>Общий прогресс</h3>
                    <p style="font-size: 1.2em;"><strong>{overall.get('success_rate', 'Не определен')}</strong></p>
                    {achievements_html}
                    {gaps_html}
                </div>

                {goals_html}

                {self._generate_metrics_progress(goal_achievement.get('metrics_progress', []))}
            </div>
        </div>
        '''

    def _generate_metrics_progress(self, metrics_progress):
        """Генерация прогресса по метрикам"""
        if not metrics_progress:
            return ''

        metrics_html = '<h3>Прогресс по метрикам успеха</h3>'

        for metric in metrics_progress:
            confidence = metric.get('confidence', 'medium')
            conf_color = self.colors['success'] if confidence == 'high' else self.colors['warning'] if confidence == 'medium' else self.colors['danger']

            # Исправляем проблемную f-строку
            actions_html = ''
            if metric.get('required_actions'):
                actions_list = ''.join([f'<li>{a}</li>' for a in metric.get('required_actions', [])])
                actions_html = f'<h5>Необходимые действия:</h5><ol>{actions_list}</ol>'

            metrics_html += f'''
            <div class="card" style="margin-bottom: 20px;">
                <h4>{metric.get('metric', '')}</h4>
                <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 20px; margin-bottom: 20px;">
                    <div>
                        <span class="metric-label">Текущее значение</span>
                        <span class="metric-value" style="display: block; color: {self.colors['danger']};">{metric.get('baseline', 'Не определено')}</span>
                    </div>
                    <div>
                        <span class="metric-label">Целевое значение</span>
                        <span class="metric-value" style="display: block; color: {self.colors['primary']};">{metric.get('target', 'Не определено')}</span>
                    </div>
                    <div>
                        <span class="metric-label">Прогноз улучшения</span>
                        <span class="metric-value" style="display: block; color: {self.colors['success']};">{metric.get('projected_improvement', 'Не определено')}</span>
                    </div>
                </div>
                <p><strong>Уверенность в прогнозе:</strong> <span class="tag" style="background: {conf_color}; color: white;">{confidence}</span></p>
                {actions_html}
            </div>
            '''

        return metrics_html

    def _generate_brief_answers_section(self, brief_answers):
        """Генерация раздела с ответами на вопросы брифа"""
        if not brief_answers or not brief_answers.get('answers'):
            return ''

        answers_html = ''
        for i, answer_data in enumerate(brief_answers.get('answers', []), 1):
            confidence = answer_data.get('confidence', 'medium')
            conf_color = self.colors['success'] if confidence == 'high' else self.colors['warning'] if confidence == 'medium' else self.colors['danger']

            # Доказательства
            evidence_html = ''
            if answer_data.get('supporting_evidence'):
                evidence_html = '<h4>Доказательства:</h4><div style="display: flex; flex-wrap: wrap; gap: 10px; margin-bottom: 20px;">'
                for e in answer_data['supporting_evidence']:
                    if isinstance(e, dict):
                        evidence_html += f'''
                        <div class="tag" style="padding: 10px 15px;">
                            <strong>{e.get('type', '')}:</strong> {e.get('content', '')}
                            <br><small>Источник: {e.get('source', '')}</small>
                        </div>
                        '''
                evidence_html += '</div>'

            # Цитаты
            quotes_html = ''
            if answer_data.get('key_quotes'):
                quotes_html = '<h4>Ключевые цитаты:</h4>'
                for quote_data in answer_data['key_quotes']:
                    if isinstance(quote_data, dict):
                        quotes_html += f'''
                        <div class="quote-card" style="margin: 15px 0;">
                            <p class="quote-text">{quote_data.get('quote', '')}</p>
                            <p style="margin-top: 10px; font-size: 0.9em; color: {self.colors['text_secondary']};">
                                <em>{quote_data.get('relevance', '')}</em>
                            </p>
                            <span class="quote-author">Интервью {quote_data.get('interview_id', '')}</span>
                        </div>
                        '''

            # Пробелы в данных
            gaps_html = ''
            if answer_data.get('data_gaps'):
                gaps_html = f'<div style="margin-top: 20px; padding: 15px; background: rgba(245, 158, 11, 0.1); border-radius: 8px;"><h5 style="color: {self.colors["warning"]};">Пробелы в данных:</h5><ul>'
                for gap in answer_data['data_gaps']:
                    gaps_html += f'<li>{gap}</li>'
                gaps_html += '</ul></div>'

            # Рекомендации - исправляем проблемную f-строку
            recommendations_html = ''
            if answer_data.get('recommendations'):
                recs_list = ''.join([f'<li>{r}</li>' for r in answer_data.get('recommendations', [])])
                recommendations_html = f'<div style="margin-top: 20px; padding: 15px; background: rgba(16, 185, 129, 0.05); border-radius: 8px;"><h5>Рекомендации:</h5><ul>{recs_list}</ul></div>'

            answers_html += f'''
            <div class="card" style="margin-bottom: 40px; border-left: 4px solid {conf_color};">
                <h3>Вопрос #{i}: {answer_data.get('question', '')}</h3>

                <div style="background: {self.colors['background']}; padding: 20px; border-radius: 8px; margin: 20px 0;">
                    <h4>Краткий ответ:</h4>
                    <p style="font-size: 1.2em; font-weight: 600;">{answer_data.get('answer_summary', '')}</p>
                </div>

                <h4>Полный ответ:</h4>
                <p style="font-size: 1.1em; line-height: 1.8; margin-bottom: 20px;">
                    {answer_data.get('answer', '')}
                </p>

                <p><strong>Уверенность в ответе:</strong> <span class="tag" style="background: {conf_color}; color: white;">{confidence}</span></p>

                {evidence_html}
                {quotes_html}
                {recommendations_html}
                {gaps_html}
            </div>
            '''

        # Кросс-вопросные инсайты
        cross_insights_html = ''
        if brief_answers.get('cross_question_insights'):
            cross_insights_html = '<h3>Инсайты, связывающие несколько вопросов</h3>'
            for insight in brief_answers['cross_question_insights']:
                cross_insights_html += f'''
                <div class="card" style="background: linear-gradient(135deg, {self.colors['background']} 0%, rgba(124, 58, 237, 0.05) 100%);">
                    <p style="font-size: 1.1em;"><strong>{insight.get('insight', '')}</strong></p>
                    <p>Связанные вопросы: {', '.join(insight.get('related_questions', []))}</p>
                    <p>Импликация: {insight.get('implication', '')}</p>
                </div>
                '''

        return f'''
        <div class="page" id="brief-answers">
            <div class="container">
                <h2>Ответы на исследовательские вопросы</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Детальные ответы на все вопросы из брифа на основе данных {brief_answers.get('total_interviews', '')} интервью
                </p>

                {answers_html}
                {cross_insights_html}

                {self._generate_unexpected_findings(brief_answers.get('unexpected_findings', []))}
            </div>
        </div>
        '''

    def _generate_unexpected_findings(self, unexpected_findings):
        """Генерация неожиданных находок"""
        if not unexpected_findings:
            return ''

        html = '<h3>Неожиданные находки</h3>'
        for finding in unexpected_findings:
            html += f'''
            <div class="card" style="background: linear-gradient(135deg, {self.colors['background']} 0%, rgba(239, 68, 68, 0.05) 100%);">
                <h4>{finding.get('finding', '')}</h4>
                <p><strong>Важность:</strong> {finding.get('importance', '')}</p>
                <p><strong>Рекомендация:</strong> {finding.get('recommendation', '')}</p>
            </div>
            '''
        return html

    def _get_professional_css(self):
        """Профессиональные CSS стили для отчета"""
        return f'''
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}

        body {{
            font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
            line-height: 1.6;
            color: {self.colors['text']};
            background: {self.colors['white']};
            font-size: 16px;
            -webkit-font-smoothing: antialiased;
        }}

        .page {{
            page-break-after: always;
            min-height: 100vh;
            padding: 60px 0;
            background: {self.colors['white']};
        }}

        .container {{
            max-width: 1200px;
            margin: 0 auto;
            padding: 0 40px;
        }}

        h1, h2, h3, h4, h5, h6 {{
            font-weight: 700;
            line-height: 1.2;
            margin-bottom: 1em;
            color: {self.colors['dark']};
        }}

        h1 {{
            font-size: 3.5em;
            font-weight: 900;
            letter-spacing: -0.02em;
        }}

        h2 {{
            font-size: 2.5em;
            font-weight: 800;
            margin-top: 2em;
            margin-bottom: 1em;
            color: {self.colors['primary']};
            position: relative;
            padding-bottom: 0.5em;
        }}

        h2:after {{
            content: '';
            position: absolute;
            bottom: 0;
            left: 0;
            width: 60px;
            height: 4px;
            background: {self.colors['primary']};
            border-radius: 2px;
        }}

        h3 {{
            font-size: 1.8em;
            font-weight: 700;
            margin-top: 1.5em;
            color: {self.colors['dark']};
        }}

        h4 {{
            font-size: 1.3em;
            font-weight: 600;
            margin-top: 1.2em;
        }}

        h5 {{
            font-size: 1.1em;
            font-weight: 600;
            color: {self.colors['text_secondary']};
        }}

        p {{
            margin-bottom: 1em;
            line-height: 1.8;
        }}

        ul, ol {{
            margin-bottom: 1em;
            padding-left: 1.5em;
        }}

        li {{
            margin-bottom: 0.5em;
            line-height: 1.8;
        }}

        .card {{
            background: {self.colors['white']};
            border-radius: 12px;
            padding: 30px;
            margin-bottom: 30px;
            box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1), 0 1px 2px rgba(0, 0, 0, 0.06);
            border: 1px solid {self.colors['border']};
            transition: all 0.3s ease;
        }}

        .card:hover {{
            box-shadow: 0 10px 15px -3px rgba(0, 0, 0, 0.1), 0 4px 6px -2px rgba(0, 0, 0, 0.05);
            transform: translateY(-2px);
        }}

        .metric-card {{
            background: linear-gradient(135deg, {self.colors['primary']} 0%, {self.colors['secondary']} 100%);
            color: {self.colors['white']};
            padding: 40px;
            border-radius: 16px;
            text-align: center;
            margin-bottom: 30px;
        }}

        .metric-value {{
            font-size: 3em;
            font-weight: 800;
            margin: 20px 0;
        }}

        .metric-label {{
            font-size: 1.1em;
            opacity: 0.9;
            font-weight: 500;
        }}

        .quote-card {{
            background: {self.colors['background']};
            border-left: 4px solid {self.colors['primary']};
            padding: 20px 30px;
            margin: 20px 0;
            font-style: italic;
            position: relative;
        }}

        .quote-card:before {{
            content: '"';
            font-size: 4em;
            color: {self.colors['primary']};
            opacity: 0.2;
            position: absolute;
            top: -10px;
            left: 10px;
        }}

        .quote-text {{
            font-size: 1.1em;
            line-height: 1.8;
            color: {self.colors['text']};
            font-style: italic;
        }}

        .quote-author {{
            display: block;
            margin-top: 15px;
            font-size: 0.9em;
            color: {self.colors['text_secondary']};
            font-style: normal;
            font-weight: 600;
        }}

        .tag {{
            display: inline-block;
            padding: 6px 16px;
            background: {self.colors['background']};
            color: {self.colors['text']};
            border-radius: 20px;
            font-size: 0.85em;
            font-weight: 500;
            margin-right: 10px;
            margin-bottom: 10px;
        }}

        .priority-high {{
            background: rgba(239, 68, 68, 0.1);
            color: {self.colors['danger']};
            font-weight: 600;
        }}

        .priority-medium {{
            background: rgba(245, 158, 11, 0.1);
            color: {self.colors['warning']};
        }}

        .priority-low {{
            background: rgba(16, 185, 129, 0.1);
            color: {self.colors['success']};
        }}

        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            background: {self.colors['white']};
            border-radius: 8px;
            overflow: hidden;
            box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1);
        }}

        th {{
            background: {self.colors['background']};
            padding: 15px;
            text-align: left;
            font-weight: 600;
            color: {self.colors['dark']};
            border-bottom: 2px solid {self.colors['border']};
        }}

        td {{
            padding: 15px;
            border-bottom: 1px solid {self.colors['border']};
        }}

        tr:hover {{
            background: {self.colors['background']};
        }}

        .persona-card {{
            background: {self.colors['white']};
            border-radius: 16px;
            padding: 40px;
            margin-bottom: 40px;
            border: 2px solid {self.colors['border']};
            position: relative;
            overflow: hidden;
        }}

        .persona-card:before {{
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            height: 6px;
            background: linear-gradient(90deg, {self.colors['primary']} 0%, {self.colors['secondary']} 100%);
        }}

        .persona-header {{
            display: flex;
            align-items: center;
            margin-bottom: 30px;
        }}

        .persona-avatar {{
            width: 100px;
            height: 100px;
            border-radius: 50%;
            background: linear-gradient(135deg, {self.colors['primary']} 0%, {self.colors['secondary']} 100%);
            display: flex;
            align-items: center;
            justify-content: center;
            color: {self.colors['white']};
            font-size: 2.5em;
            font-weight: 700;
            margin-right: 30px;
        }}

        .pattern-card {{
            background: linear-gradient(135deg, {self.colors['background']} 0%, rgba(37, 99, 235, 0.05) 100%);
            border-radius: 12px;
            padding: 30px;
            margin-bottom: 30px;
            border-left: 4px solid {self.colors['primary']};
        }}

        .timeline {{
            position: relative;
            padding-left: 40px;
        }}

        .timeline:before {{
            content: '';
            position: absolute;
            left: 15px;
            top: 0;
            bottom: 0;
            width: 2px;
            background: {self.colors['border']};
        }}

        .timeline-item {{
            position: relative;
            margin-bottom: 30px;
        }}

        .timeline-item:before {{
            content: '';
            position: absolute;
            left: -27px;
            top: 5px;
            width: 12px;
            height: 12px;
            border-radius: 50%;
            background: {self.colors['primary']};
            border: 3px solid {self.colors['white']};
            box-shadow: 0 0 0 4px {self.colors['background']};
        }}

        .roadmap-phase {{
            background: {self.colors['white']};
            border-radius: 12px;
            padding: 30px;
            margin-bottom: 20px;
            border: 2px solid {self.colors['border']};
            position: relative;
        }}

        .phase-number {{
            position: absolute;
            top: -15px;
            left: 30px;
            background: {self.colors['primary']};
            color: {self.colors['white']};
            width: 40px;
            height: 40px;
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            font-weight: 700;
            font-size: 1.2em;
        }}

        .stats-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
            margin: 30px 0;
        }}

        .stat-card {{
            background: {self.colors['background']};
            padding: 30px;
            border-radius: 12px;
            text-align: center;
        }}

        .stat-number {{
            font-size: 2.5em;
            font-weight: 800;
            color: {self.colors['primary']};
            margin: 10px 0;
        }}

        .chart-container {{
            margin: 30px 0;
            padding: 20px;
            background: {self.colors['white']};
            border-radius: 12px;
            box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1);
        }}

        .pain-point-card {{
            background: {self.colors['white']};
            border-radius: 12px;
            padding: 30px;
            margin-bottom: 30px;
            border-left: 4px solid {self.colors['danger']};
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.05);
        }}

        .insight-card {{
            background: linear-gradient(135deg, rgba(124, 58, 237, 0.05) 0%, rgba(37, 99, 235, 0.05) 100%);
            border-radius: 12px;
            padding: 30px;
            margin-bottom: 30px;
            border: 1px solid rgba(124, 58, 237, 0.2);
        }}

        .recommendation-card {{
            background: {self.colors['white']};
            border-radius: 16px;
            padding: 40px;
            margin-bottom: 30px;
            border: 2px solid {self.colors['success']};
            position: relative;
        }}

        .recommendation-card:before {{
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            height: 4px;
            background: {self.colors['success']};
        }}

        .cover-page {{
            min-height: 100vh;
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            text-align: center;
            background: linear-gradient(135deg, {self.colors['primary']} 0%, {self.colors['secondary']} 100%);
            color: {self.colors['white']};
            position: relative;
            overflow: hidden;
        }}

        .cover-page:before {{
            content: '';
            position: absolute;
            top: -50%;
            right: -50%;
            width: 200%;
            height: 200%;
            background: radial-gradient(circle, rgba(255,255,255,0.1) 0%, transparent 70%);
            animation: pulse 4s ease-in-out infinite;
        }}

        @keyframes pulse {{
            0%, 100% {{ transform: scale(1); opacity: 0.5; }}
            50% {{ transform: scale(1.1); opacity: 0.3; }}
        }}

        .toc {{
            background: {self.colors['background']};
            border-radius: 16px;
            padding: 40px;
            margin: 40px 0;
        }}

        .toc-item {{
            display: flex;
            justify-content: space-between;
            padding: 15px 0;
            border-bottom: 1px solid {self.colors['border']};
            color: {self.colors['text']};
            text-decoration: none;
            transition: all 0.3s ease;
        }}

        .toc-item:hover {{
            color: {self.colors['primary']};
            padding-left: 10px;
        }}

        @media print {{
            .page {{ page-break-after: always; }}
            body {{ font-size: 14px; }}
            .card {{ box-shadow: none; }}
        }}
        '''

    def _generate_cover_page(self):
        """Генерация титульной страницы"""
        current_date = datetime.now().strftime("%d.%m.%Y")
        return f'''
        <div class="cover-page">
            <div style="position: relative; z-index: 1;">
                <h1 style="font-size: 4em; margin-bottom: 0.5em;">{self.config.report_title}</h1>
                <p style="font-size: 1.5em; opacity: 0.9; margin-bottom: 2em;">Комплексный анализ пользовательского опыта</p>
                <div style="margin-top: 4em;">
                    <p style="font-size: 1.2em; opacity: 0.8;">{self.config.name}</p>
                    <p style="opacity: 0.7;">{current_date}</p>
                </div>
            </div>
        </div>
        '''

    def _generate_table_of_contents(self, analysis_data):
        """Генерация оглавления"""
        # Определяем какие разделы есть
        has_brief = analysis_data.get('brief_data') is not None
        has_segments = len(analysis_data.get('base_analysis', {}).get('segments', [])) > 0
        has_personas = len(analysis_data.get('personas', [])) > 0
        has_brief_answers = bool(analysis_data.get('brief_answers', {}).get('answers', []))
        has_goal_achievement = bool(analysis_data.get('goal_achievement', {}).get('goals', []))

        toc_items = []

        if has_brief:
            toc_items.append(('Контекст исследования', 'brief'))

        toc_items.append(('Общий обзор', 'overview'))

        if has_goal_achievement:
            toc_items.append(('Достижение целей', 'goal-achievement'))

        if has_brief_answers:
            toc_items.append(('Ответы на вопросы', 'brief-answers'))

        toc_items.extend([
            ('Текущее состояние', 'current-state'),
            ('Ключевые метрики', 'key-metrics')
        ])

        if has_segments:
            toc_items.append(('Сегменты пользователей', 'segments'))

        if has_personas:
            toc_items.append(('Персоны', 'personas'))

        toc_items.extend([
            ('Ключевые проблемы', 'pain-points'),
            ('Поведенческие паттерны', 'patterns'),
            ('Эмоциональный опыт', 'emotions'),
            ('Ключевые инсайты', 'insights'),
            ('Противоречия', 'contradictions'),
            ('Важные цитаты', 'quotes'),
            ('Рекомендации', 'recommendations'),
            ('Матрица приоритетов', 'priority-matrix'),
            ('Дорожная карта', 'roadmap'),
            ('Приложение', 'appendix')
        ])

        toc_html = ''
        for i, (title, anchor) in enumerate(toc_items, 1):
            toc_html += f'''
            <a href="#{anchor}" class="toc-item">
                <span>{i}. {title}</span>
                <span style="opacity: 0.5;">→</span>
            </a>
            '''

        return f'''
        <div class="page">
            <div class="container">
                <h2>Содержание</h2>
                <div class="toc">
                    {toc_html}
                </div>
            </div>
        </div>
        '''

    def _generate_overview_section(self, analysis_data, charts):
        """Генерация обзорной секции"""
        defense = analysis_data.get('defense_materials', {})
        total_interviews = analysis_data.get('total_interviews', 0)
        findings = analysis_data.get('findings', None)

        exec_summary = defense.get('executive_summary', '')
        if not exec_summary and findings:
            exec_summary = findings.executive_summary

        return f'''
        <div class="page" id="overview">
            <div class="container">
                <h2>Общий обзор исследования</h2>

                <div class="card" style="background: linear-gradient(135deg, {self.colors['background']} 0%, rgba(37, 99, 235, 0.05) 100%);">
                    <h3>Executive Summary</h3>
                    <p style="font-size: 1.2em; line-height: 1.8;">
                        {exec_summary}
                    </p>
                </div>

                <div class="stats-grid">
                    <div class="stat-card">
                        <div class="stat-number">{total_interviews}</div>
                        <div class="metric-label">Интервью проведено</div>
                    </div>
                    <div class="stat-card">
                        <div class="stat-number">{len(analysis_data.get('base_analysis', {}).get('problems', []))}</div>
                        <div class="metric-label">Ключевых проблем</div>
                    </div>
                    <div class="stat-card">
                        <div class="stat-number">{len(analysis_data.get('base_analysis', {}).get('segments', []))}</div>
                        <div class="metric-label">Сегментов выявлено</div>
                    </div>
                    <div class="stat-card">
                        <div class="stat-number">{len(analysis_data.get('recommendations', {}).get('quick_wins', []))}</div>
                        <div class="metric-label">Quick wins</div>
                    </div>
                </div>

                {charts.get('overview_chart', '')}
            </div>
        </div>
        '''

    def _generate_current_state_section(self, metrics, charts):
        """Генерация секции текущего состояния"""
        nps_color = self.colors['danger']
        if isinstance(metrics.get('estimated_nps'), int):
            if metrics['estimated_nps'] > 0:
                nps_color = self.colors['warning']
            if metrics['estimated_nps'] > 30:
                nps_color = self.colors['success']

        churn_color = self.colors['danger']
        if metrics.get('churn_risk') == 'Средний':
            churn_color = self.colors['warning']
        elif metrics.get('churn_risk') == 'Низкий':
            churn_color = self.colors['success']

        return f'''
        <div class="page" id="current-state">
            <div class="container">
                <h2>Текущее состояние продукта</h2>

                <div class="metric-card">
                    <h3 style="color: white; margin-bottom: 30px;">Ключевые метрики на основе анализа</h3>
                    <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 40px;">
                        <div>
                            <div class="metric-label">Предполагаемый NPS</div>
                            <div class="metric-value" style="color: {nps_color};">
                                {metrics.get('estimated_nps', 'Н/Д')}
                            </div>
                        </div>
                        <div>
                            <div class="metric-label">Риск оттока</div>
                            <div class="metric-value" style="color: {churn_color};">
                                {metrics.get('churn_risk', 'Не определен')}
                            </div>
                        </div>
                    </div>
                </div>

                <div class="stats-grid">
                    <div class="card">
                        <h4>Среднее количество проблем</h4>
                        <div style="font-size: 2em; font-weight: 700; color: {self.colors['danger']};">
                            {metrics.get('avg_pains_per_user', 0)}
                        </div>
                        <p style="color: {self.colors['text_secondary']};">на пользователя</p>
                    </div>
                    <div class="card">
                        <h4>Среднее количество потребностей</h4>
                        <div style="font-size: 2em; font-weight: 700; color: {self.colors['primary']};">
                            {metrics.get('avg_needs_per_user', 0)}
                        </div>
                        <p style="color: {self.colors['text_secondary']};">на пользователя</p>
                    </div>
                    <div class="card">
                        <h4>Негативные эмоции</h4>
                        <div style="font-size: 2em; font-weight: 700; color: {self.colors['warning']};">
                            {metrics.get('negative_emotion_ratio', 0):.0f}%
                        </div>
                        <p style="color: {self.colors['text_secondary']};">от всех эмоций</p>
                    </div>
                </div>

                {charts.get('metrics_chart', '')}

                <div class="card" style="background: rgba(239, 68, 68, 0.05); border-left: 4px solid {self.colors['danger']};">
                    <h4>Что это означает?</h4>
                    <p>На основе анализа {metrics.get('sample_size', 0)} интервью и {metrics.get('total_emotions_analyzed', 0)} эмоциональных моментов,
                    текущий пользовательский опыт характеризуется значительным количеством проблем и неудовлетворенных потребностей.
                    Это создает риски для удержания пользователей и требует срочных действий.</p>
                </div>
            </div>
        </div>
        '''

    def _generate_key_metrics_section(self, analysis_data, charts):
        """Генерация секции ключевых метрик"""
        defense = analysis_data.get('defense_materials', {})
        key_findings = defense.get('key_findings', [])

        findings_html = ''
        for i, finding in enumerate(key_findings[:5], 1):
            findings_html += f'''
            <div class="card" style="border-left: 4px solid {self.colors['primary']};">
                <h4>Находка #{i}</h4>
                <p style="font-size: 1.1em; line-height: 1.8;">{finding}</p>
            </div>
            '''

        return f'''
        <div class="page" id="key-metrics">
            <div class="container">
                <h2>Ключевые находки исследования</h2>

                {findings_html}

                {charts.get('findings_chart', '')}
            </div>
        </div>
        '''

    def _generate_user_segments_section(self, segments, charts):
        """Генерация секции сегментов пользователей"""
        segments_html = ''

        colors = [self.colors['primary'], self.colors['secondary'], self.colors['success'],
                 self.colors['warning'], self.colors['danger']]

        for i, segment in enumerate(segments[:5]):
            color = colors[i % len(colors)]

            # Демография
            demographics = segment.get('demographics', {})
            demo_html = '<div style="margin: 20px 0;"><h5>Демография:</h5><ul>'
            if demographics.get('age_range'):
                demo_html += f'<li>Возраст: {demographics["age_range"]}</li>'
            if demographics.get('occupation_types'):
                demo_html += f'<li>Профессии: {", ".join(demographics["occupation_types"][:3])}</li>'
            if demographics.get('location'):
                demo_html += f'<li>Локация: {demographics["location"]}</li>'
            demo_html += '</ul></div>'

            # Поведенческие черты
            behavioral = segment.get('behavioral_traits', {})
            behavioral_html = ''
            if behavioral:
                behavioral_html = '<div style="margin: 20px 0;"><h5>Поведение:</h5><ul>'
                if behavioral.get('usage_patterns'):
                    for pattern in behavioral['usage_patterns'][:3]:
                        behavioral_html += f'<li>{pattern}</li>'
                behavioral_html += '</ul></div>'

            # Боли и потребности
            pains_html = ''
            if segment.get('pain_points'):
                pains_html = '<div style="margin: 20px 0;"><h5>Основные проблемы:</h5><ul>'
                for pain in segment['pain_points'][:3]:
                    pains_html += f'<li>{pain}</li>'
                pains_html += '</ul></div>'

            # Цитаты
            quotes_html = ''
            if segment.get('representative_quotes'):
                quotes_html = '<div style="margin-top: 30px;"><h5>Характерные высказывания:</h5>'
                for quote in segment['representative_quotes'][:2]:
                    quotes_html += f'''
                    <div class="quote-card" style="margin: 15px 0;">
                        <p class="quote-text">{quote}</p>
                    </div>
                    '''
                quotes_html += '</div>'

            segments_html += f'''
            <div class="card" style="border-top: 4px solid {color};">
                <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 20px;">
                    <h3>{segment.get('name', f'Сегмент {i+1}')}</h3>
                    <span class="tag" style="background: {color}; color: white; font-size: 1em; padding: 10px 20px;">
                        {segment.get('size', 'Размер не определен')}
                    </span>
                </div>

                <p style="font-size: 1.1em; line-height: 1.8; margin-bottom: 20px;">
                    {segment.get('description', '')}
                </p>

                {demo_html}
                {behavioral_html}
                {pains_html}

                <div style="margin-top: 20px;">
                    <span class="tag">Интервью: {', '.join(map(str, segment.get('interview_ids', [])[:5]))}</span>
                </div>

                {quotes_html}
            </div>
            '''

        return f'''
        <div class="page" id="segments">
            <div class="container">
                <h2>Сегменты пользователей</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    На основе анализа выявлено {len(segments)} ключевых сегментов пользователей
                </p>

                {segments_html}

                {charts.get('segments_chart', '')}
            </div>
        </div>
        '''

    def _generate_personas_section(self, personas):
        """Генерация секции персон"""
        personas_html = ''

        for i, persona in enumerate(personas[:4]):
            # Демография
            demo = persona.get('demographics', {})
            demo_html = f'''
            <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px; margin: 20px 0;">
                <div><strong>Возраст:</strong> {demo.get('age', 'Н/Д')}</div>
                <div><strong>Профессия:</strong> {demo.get('occupation', 'Н/Д')}</div>
                <div><strong>Локация:</strong> {demo.get('location', 'Н/Д')}</div>
            </div>
            '''

            # Цели и фрустрации
            goals_html = '<div style="margin: 20px 0;"><h5>Цели:</h5><ul>'
            for goal in persona.get('goals', [])[:3]:
                goals_html += f'<li>{goal}</li>'
            goals_html += '</ul></div>'

            frustrations_html = '<div style="margin: 20px 0;"><h5>Фрустрации:</h5><ul>'
            for frustration in persona.get('frustrations', [])[:3]:
                frustrations_html += f'<li>{frustration}</li>'
            frustrations_html += '</ul></div>'

            # Цитаты
            quotes_html = ''
            if persona.get('real_quotes'):
                quotes_html = '<div style="margin-top: 30px;"><h5>Реальные цитаты:</h5>'
                for quote in persona['real_quotes'][:3]:
                    quotes_html += f'''
                    <div class="quote-card" style="margin: 15px 0;">
                        <p class="quote-text">{quote}</p>
                    </div>
                    '''
                quotes_html += '</div>'

            # Типичный сценарий
            scenario_html = ''
            if persona.get('typical_scenario'):
                scenario_html = f'''
                <div class="card" style="background: {self.colors['background']}; margin-top: 20px;">
                    <h5>Типичный сценарий:</h5>
                    <p>{persona['typical_scenario']}</p>
                </div>
                '''

            # ИСПРАВЛЕНИЕ: преобразуем map в list перед использованием среза
            based_on_interviews = list(map(str, persona.get('based_on_interviews', [])))[:3]

            personas_html += f'''
            <div class="persona-card">
                <div class="persona-header">
                    <div class="persona-avatar">{persona.get('name', 'Персона')[0]}</div>
                    <div>
                        <h3>{persona.get('name', f'Персона {i+1}')}</h3>
                        <p style="font-size: 1.2em; color: {self.colors['text_secondary']}; font-style: italic;">
                            "{persona.get('tagline', '')}"
                        </p>
                        <span class="tag">Основана на интервью: {', '.join(based_on_interviews)}</span>
                    </div>
                </div>

                <p style="font-size: 1.1em; line-height: 1.8; margin-bottom: 30px;">
                    {persona.get('description', '')}
                </p>

                {demo_html}

                <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 30px;">
                    {goals_html}
                    {frustrations_html}
                </div>

                {scenario_html}
                {quotes_html}
            </div>
            '''

        return f'''
        <div class="page" id="personas">
            <div class="container">
                <h2>Персоны пользователей</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Детальные портреты основанные на реальных данных интервью
                </p>

                {personas_html}
            </div>
        </div>
        '''

    def _generate_pain_points_section_full(self, problems, charts):
        """Генерация полной секции болевых точек"""
        if not problems:
            return ''

        problems_html = ''

        severity_colors = {
            'critical': self.colors['danger'],
            'high': self.colors['warning'],
            'medium': self.colors['primary'],
            'low': self.colors['success']
        }

        for i, problem in enumerate(problems[:10], 1):
            severity = problem.get('severity', 'medium')
            color = severity_colors.get(severity, self.colors['primary'])

            # Доказательства
            evidence_html = ''
            if problem.get('evidence'):
                evidence_html = '<div style="margin: 20px 0;"><h5>Доказательства:</h5><ul>'
                for evidence in problem['evidence'][:3]:
                    evidence_html += f'<li>{evidence}</li>'
                evidence_html += '</ul></div>'

            # Цитаты
            quotes_html = ''
            if problem.get('quotes'):
                quotes_html = '<div style="margin-top: 20px;"><h5>Цитаты пользователей:</h5>'
                for quote_data in problem['quotes'][:2]:
                    if isinstance(quote_data, dict):
                        quotes_html += f'''
                        <div class="quote-card" style="margin: 15px 0;">
                            <p class="quote-text">{quote_data.get('text', '')}</p>
                            <span class="quote-author">Интервью {quote_data.get('interview_id', '')}</span>
                        </div>
                        '''
                    else:
                        quotes_html += f'''
                        <div class="quote-card" style="margin: 15px 0;">
                            <p class="quote-text">{quote_data}</p>
                        </div>
                        '''
                quotes_html += '</div>'

            # Возможность
            opportunity_html = ''
            opportunity = problem.get('opportunity', {})
            if opportunity and isinstance(opportunity, dict):
                opportunity_html = f'''
                <div class="card" style="background: rgba(16, 185, 129, 0.05); margin-top: 20px;">
                    <h5>Возможность:</h5>
                    <p>{opportunity.get('description', '')}</p>
                    {f'<p><strong>Ценностное предложение:</strong> {opportunity.get("value_prop", "")}</p>' if opportunity.get('value_prop') else ''}
                </div>
                '''

            problems_html += f'''
            <div class="pain-point-card">
                <div style="display: flex; justify-content: space-between; align-items: start; margin-bottom: 20px;">
                    <h3>#{i}. {problem.get('title', problem.get('problem_title', 'Проблема'))}</h3>
                    <div>
                        <span class="tag priority-{severity}" style="font-size: 0.9em;">
                            {severity.upper()}
                        </span>
                        <span class="tag" style="margin-left: 10px;">
                            {problem.get('affected_percentage', 'Н/Д')}
                        </span>
                    </div>
                </div>

                <p style="font-size: 1.1em; line-height: 1.8; margin-bottom: 20px;">
                    {problem.get('description', problem.get('problem_description', ''))}
                </p>

                {evidence_html}
                {quotes_html}
                {opportunity_html}

                <div style="margin-top: 20px; display: flex; gap: 15px;">
                    <span class="tag">Приоритет: {problem.get('priority', 'P2')}</span>
                    <span class="tag">Усилия: {problem.get('effort', 'M')}</span>
                </div>
            </div>
            '''

        return f'''
        <div class="page" id="pain-points">
            <div class="container">
                <h2>Ключевые проблемы пользователей</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Приоритизированный список проблем на основе частоты упоминания и критичности
                </p>

                {problems_html}

                {charts.get('problems_chart', '')}
            </div>
        </div>
        '''

    def _generate_behavioral_patterns_section_full(self, patterns):
        """Генерация полной секции поведенческих паттернов"""
        if not patterns:
            return ''

        patterns_html = ''

        for i, pattern in enumerate(patterns[:8], 1):
            # Доказательства
            evidence_html = ''
            if pattern.get('evidence'):
                evidence_html = '<div style="margin: 20px 0;"><h5>Доказательства:</h5>'
                for e in pattern['evidence'][:3]:
                    if isinstance(e, dict):
                        evidence_html += f'''
                        <div class="card" style="margin: 10px 0; background: {self.colors['background']};">
                            <p>{e.get('content', '')}</p>
                            <span class="tag">Интервью: {', '.join(map(str, e.get('interview_ids', [])))}</span>
                        </div>
                        '''
                evidence_html += '</div>'

            # Триггеры
            triggers_html = ''
            if pattern.get('triggers'):
                triggers_html = '<div style="margin: 20px 0;"><h5>Триггеры:</h5><ul>'
                for trigger in pattern['triggers'][:3]:
                    if isinstance(trigger, dict):
                        triggers_html += f'<li><strong>{trigger.get("trigger", "")}:</strong> {trigger.get("type", "")} (надежность: {trigger.get("reliability", "")})</li>'
                    else:
                        triggers_html += f'<li>{trigger}</li>'
                triggers_html += '</ul></div>'

            # Эмоциональный путь
            emotional_html = ''
            if pattern.get('emotional_journey'):
                emotional_html = '<div style="margin: 20px 0;"><h5>Эмоциональный путь:</h5>'
                emotional_html += '<div class="timeline">'
                for stage in pattern['emotional_journey'][:4]:
                    if isinstance(stage, dict):
                        emotional_html += f'''
                        <div class="timeline-item">
                            <h6>{stage.get('stage', '')}</h6>
                            <p>{stage.get('emotion', '')} (интенсивность: {stage.get('intensity', 0)}/10)</p>
                        </div>
                        '''
                emotional_html += '</div></div>'

            # Импликации для дизайна
            design_html = ''
            design_impl = pattern.get('design_implications', {})
            if design_impl and isinstance(design_impl, dict):
                support_html = f'<p><strong>Поддержать паттерн:</strong> {design_impl.get("support_pattern", "")}</p>' if design_impl.get('support_pattern') else ''
                break_html = f'<p><strong>Изменить паттерн:</strong> {design_impl.get("break_pattern", "")}</p>' if design_impl.get('break_pattern') else ''
                intervention_html = ''
                if design_impl.get('intervention_points'):
                    intervention_html = f'<p><strong>Точки вмешательства:</strong> {", ".join(design_impl.get("intervention_points", []))}</p>'

                design_html = f'''
                <div class="card" style="background: rgba(37, 99, 235, 0.05); margin-top: 20px;">
                    <h5>Импликации для дизайна:</h5>
                    {support_html}
                    {break_html}
                    {intervention_html}
                </div>
                '''

            # Характерные цитаты - исправляем проблемную f-строку
            quotes_html = ''
            if pattern.get('representative_quotes'):
                quotes_items = ''.join([f'<div class="quote-card" style="margin: 15px 0;"><p class="quote-text">{q}</p></div>' for q in pattern.get('representative_quotes', [])[:2]])
                quotes_html = f'<div style="margin-top: 20px;"><h5>Характерные цитаты:</h5>{quotes_items}</div>'

            patterns_html += f'''
            <div class="pattern-card">
                <div style="display: flex; justify-content: space-between; align-items: start; margin-bottom: 20px;">
                    <h3>{pattern.get('pattern', f'Паттерн {i}')}</h3>
                    <div>
                        <span class="tag">{pattern.get('frequency', '')}</span>
                        <span class="tag" style="margin-left: 10px;">
                            Сила: {pattern.get('strength', 'moderate')}
                        </span>
                    </div>
                </div>

                <p style="font-size: 1.1em; line-height: 1.8; margin-bottom: 20px;">
                    {pattern.get('description', '')}
                </p>

                {evidence_html}
                {triggers_html}
                {emotional_html}
                {design_html}
                {quotes_html}
            </div>
            '''

        return f'''
        <div class="page" id="patterns">
            <div class="container">
                <h2>Поведенческие паттерны</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Устойчивые модели поведения, выявленные в процессе анализа
                </p>

                {patterns_html}
            </div>
        </div>
        '''

    def _generate_emotional_journey_section(self, summaries):
        """Генерация секции эмоционального пути"""
        # Собираем все эмоциональные моменты
        all_emotions = []
        for summary in summaries:
            for emotion in summary.emotional_journey[:5]:
                if isinstance(emotion, dict):
                    all_emotions.append({
                        'interview_id': summary.interview_id,
                        'moment': emotion.get('moment', ''),
                        'emotion': emotion.get('emotion', ''),
                        'trigger': emotion.get('trigger', ''),
                        'intensity': emotion.get('intensity', 0),
                        'quote': emotion.get('quote', '')
                    })

        # Группируем по типам эмоций
        positive_emotions = [e for e in all_emotions if any(pos in e['emotion'].lower() for pos in ['радость', 'удовлетворение', 'восторг', 'счастье', 'довольство'])]
        negative_emotions = [e for e in all_emotions if any(neg in e['emotion'].lower() for neg in ['фрустрация', 'злость', 'разочарование', 'раздражение', 'страх'])]

        # Находим пиковые моменты
        peak_emotions = sorted(all_emotions, key=lambda x: x['intensity'], reverse=True)[:5]

        emotions_html = ''

        # Пиковые эмоции
        if peak_emotions:
            emotions_html += '<h3>Эмоциональные пики</h3>'
            for emotion in peak_emotions:
                intensity_color = self.colors['danger'] if emotion['intensity'] >= 8 else self.colors['warning'] if emotion['intensity'] >= 5 else self.colors['primary']

                emotions_html += f'''
                <div class="card" style="border-left: 4px solid {intensity_color};">
                    <div style="display: flex; justify-content: space-between; align-items: start;">
                        <div style="flex: 1;">
                            <h4>{emotion['moment']}</h4>
                            <p><strong>Триггер:</strong> {emotion['trigger']}</p>
                            <p><strong>Эмоция:</strong> {emotion['emotion']}</p>
                        </div>
                        <div style="text-align: center; margin-left: 30px;">
                            <div style="font-size: 2em; font-weight: 700; color: {intensity_color};">
                                {emotion['intensity']}/10
                            </div>
                            <span class="metric-label">Интенсивность</span>
                        </div>
                    </div>
                    {f'<div class="quote-card" style="margin-top: 20px;"><p class="quote-text">{emotion["quote"]}</p><span class="quote-author">Интервью {emotion["interview_id"]}</span></div>' if emotion['quote'] else ''}
                </div>
                '''

        # Позитивные моменты
        if positive_emotions:
            emotions_html += f'''
            <h3 style="margin-top: 40px;">Позитивные моменты</h3>
            <div class="card" style="background: rgba(16, 185, 129, 0.05);">
                <p>Выявлено <strong>{len(positive_emotions)}</strong> позитивных эмоциональных моментов</p>
                <ul>
            '''
            for emotion in positive_emotions[:3]:
                emotions_html += f'<li><strong>{emotion["moment"]}:</strong> {emotion["emotion"]} (триггер: {emotion["trigger"]})</li>'
            emotions_html += '</ul></div>'

        # Негативные моменты
        if negative_emotions:
            emotions_html += f'''
            <h3 style="margin-top: 40px;">Негативные моменты</h3>
            <div class="card" style="background: rgba(239, 68, 68, 0.05);">
                <p>Выявлено <strong>{len(negative_emotions)}</strong> негативных эмоциональных моментов</p>
                <ul>
            '''
            for emotion in negative_emotions[:3]:
                emotions_html += f'<li><strong>{emotion["moment"]}:</strong> {emotion["emotion"]} (триггер: {emotion["trigger"]})</li>'
            emotions_html += '</ul></div>'

        return f'''
        <div class="page" id="emotions">
            <div class="container">
                <h2>Эмоциональный опыт пользователей</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Анализ эмоциональных реакций и их триггеров
                </p>

                {emotions_html}
            </div>
        </div>
        '''

    def _generate_insights_section_full(self, insights):
        """Генерация полной секции инсайтов"""
        if not insights:
            return ''

        insights_html = ''

        for i, insight in enumerate(insights[:8], 1):
            # Доказательства
            evidence_html = ''
            if insight.get('evidence'):
                evidence_html = '<div style="margin: 20px 0;"><h5>Основано на:</h5><ul>'
                for evidence in insight['evidence'][:4]:
                    evidence_html += f'<li>{evidence}</li>'
                evidence_html += '</ul></div>'

            # Возможности
            opportunity_html = ''
            opportunity = insight.get('opportunity')
            if opportunity:
                if isinstance(opportunity, str):
                    opportunity_html = f'<p><strong>Возможность:</strong> {opportunity}</p>'
                elif isinstance(opportunity, dict):
                    opportunity_html = f'<p><strong>Возможность:</strong> {opportunity.get("description", "")}</p>'

            # Цитаты
            quotes_html = ''
            if insight.get('quotes'):
                quotes_html = '<div style="margin-top: 20px;">'
                for quote_data in insight['quotes'][:2]:
                    if isinstance(quote_data, dict):
                        quotes_html += f'''
                        <div class="quote-card">
                            <p class="quote-text">{quote_data.get('text', '')}</p>
                            <span class="quote-author">Интервью {quote_data.get('interview_id', '')}</span>
                        </div>
                        '''
                    else:
                        quotes_html += f'''
                        <div class="quote-card">
                            <p class="quote-text">{quote_data}</p>
                        </div>
                        '''
                quotes_html += '</div>'

            insights_html += f'''
            <div class="insight-card">
                <h3>Инсайт #{i}: {insight.get('title', '')}</h3>

                <p style="font-size: 1.2em; line-height: 1.8; margin: 20px 0; font-weight: 500;">
                    {insight.get('description', '')}
                </p>

                {evidence_html}
                {opportunity_html}
                {quotes_html}

                <div style="margin-top: 20px; display: flex; gap: 15px;">
                    <span class="tag priority-{insight.get('severity', 'medium')}">
                        {insight.get('severity', 'medium').upper()}
                    </span>
                    <span class="tag">Приоритет: {insight.get('priority', 'P2')}</span>
                </div>
            </div>
            '''

        return f'''
        <div class="page" id="insights">
            <div class="container">
                <h2>Ключевые инсайты</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Глубокие выводы, основанные на кросс-анализе всех интервью
                </p>

                {insights_html}
            </div>
        </div>
        '''

    def _generate_contradictions_section(self, summaries):
        """Генерация секции противоречий"""
        # Собираем все противоречия
        all_contradictions = []
        for summary in summaries:
            for contradiction in summary.contradictions[:3]:
                if isinstance(contradiction, str):
                    all_contradictions.append({
                        'interview_id': summary.interview_id,
                        'contradiction': contradiction,
                        'type': 'simple'
                    })

        # Также проверяем более сложные противоречия из findings
        contradictions_html = ''

        if not all_contradictions:
            contradictions_html = '''
            <div class="card">
                <p>Значительных противоречий в ответах респондентов не выявлено,
                что говорит о консистентности пользовательского опыта.</p>
            </div>
            '''
        else:
            for i, contra in enumerate(all_contradictions[:6], 1):
                contradictions_html += f'''
                <div class="card" style="border-left: 4px solid {self.colors['warning']};">
                    <h4>Противоречие #{i}</h4>
                    <p>{contra['contradiction']}</p>
                    <span class="tag">Интервью {contra['interview_id']}</span>
                </div>
                '''

        return f'''
        <div class="page" id="contradictions">
            <div class="container">
                <h2>Противоречия и неоднозначности</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Выявленные несоответствия в ответах респондентов
                </p>

                {contradictions_html}
            </div>
        </div>
        '''

    def _generate_quotes_section(self, summaries):
        """Генерация секции важных цитат"""
        # Собираем самые мощные цитаты
        power_quotes = []
        for summary in summaries:
            for quote in summary.quotes[:5]:
                if isinstance(quote, dict):
                    quote_data = {
                        'interview_id': summary.interview_id,
                        'text': quote.get('text', ''),
                        'significance': quote.get('significance', ''),
                        'quote_type': quote.get('quote_type', 'general'),
                        'context': quote.get('context', '')
                    }
                    if len(quote_data['text']) > 50:  # Только длинные цитаты
                        power_quotes.append(quote_data)

        # Группируем по типам
        pain_quotes = [q for q in power_quotes if q['quote_type'] == 'pain']
        need_quotes = [q for q in power_quotes if q['quote_type'] == 'need']
        insight_quotes = [q for q in power_quotes if q['quote_type'] == 'insight']
        emotion_quotes = [q for q in power_quotes if q['quote_type'] == 'emotion']

        quotes_html = ''

        # Цитаты о проблемах
        if pain_quotes:
            quotes_html += '<h3>Цитаты о проблемах</h3>'
            for quote in pain_quotes[:3]:
                quotes_html += f'''
                <div class="quote-card" style="border-left-color: {self.colors['danger']};">
                    <p class="quote-text">{quote['text']}</p>
                    {f'<p style="margin: 10px 0; font-size: 0.9em; color: {self.colors["text_secondary"]};">{quote["context"]}</p>' if quote['context'] else ''}
                    <span class="quote-author">Интервью {quote['interview_id']}</span>
                </div>
                '''

        # Цитаты о потребностях
        if need_quotes:
            quotes_html += '<h3 style="margin-top: 40px;">Цитаты о потребностях</h3>'
            for quote in need_quotes[:3]:
                quotes_html += f'''
                <div class="quote-card" style="border-left-color: {self.colors['primary']};">
                    <p class="quote-text">{quote['text']}</p>
                    <span class="quote-author">Интервью {quote['interview_id']}</span>
                </div>
                '''

        # Эмоциональные цитаты
        if emotion_quotes:
            quotes_html += '<h3 style="margin-top: 40px;">Эмоциональные высказывания</h3>'
            for quote in emotion_quotes[:3]:
                quotes_html += f'''
                <div class="quote-card" style="border-left-color: {self.colors['secondary']};">
                    <p class="quote-text">{quote['text']}</p>
                    <span class="quote-author">Интервью {quote['interview_id']}</span>
                </div>
                '''

        # Если цитат мало, добавляем общие
        if len(power_quotes) < 5:
            quotes_html += '<h3 style="margin-top: 40px;">Дополнительные важные высказывания</h3>'
            for quote in power_quotes[:5]:
                quotes_html += f'''
                <div class="quote-card">
                    <p class="quote-text">{quote['text']}</p>
                    {f'<p style="margin: 10px 0; font-size: 0.9em; color: {self.colors["text_secondary"]}; font-style: normal;"><strong>Значимость:</strong> {quote["significance"]}</p>' if quote['significance'] else ''}
                    <span class="quote-author">Интервью {quote['interview_id']}</span>
                </div>
                '''

        return f'''
        <div class="page" id="quotes">
            <div class="container">
                <h2>Важные цитаты респондентов</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Наиболее значимые высказывания, раскрывающие суть пользовательского опыта
                </p>

                {quotes_html}
            </div>
        </div>
        '''

    def _generate_recommendations_section_full(self, recommendations):
        """Генерация полной секции рекомендаций"""
        recs_html = ''

        # Quick wins
        if recommendations.get('quick_wins'):
            recs_html += '<h3>Quick Wins (быстрые победы)</h3>'
            for i, rec in enumerate(recommendations['quick_wins'][:5], 1):
                # Исправляем проблемные f-строки
                steps_html = ''.join([f'<li>{step}</li>' for step in rec.get('implementation_steps', [])])
                problems_html = ''.join([f'<li>{problem}</li>' for problem in rec.get('affected_problems', [])[:3]])
                metrics_tags = ''.join([f'<span class="tag">Метрика: {metric}</span>' for metric in rec.get('success_metrics', [])[:2]])

                # Цитата поддержки
                quote_html = ''
                if rec.get('user_quotes_supporting'):
                    quote_html = f'<div class="quote-card" style="margin-top: 20px;"><p class="quote-text">{rec.get("user_quotes_supporting", [""])[0]}</p></div>'

                recs_html += f'''
                <div class="recommendation-card">
                    <h4>#{i}. {rec.get('title', '')}</h4>

                    <p style="font-size: 1.1em; line-height: 1.8; margin: 20px 0;">
                        {rec.get('description', '')}
                    </p>

                    <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 30px; margin: 30px 0;">
                        <div>
                            <h5>Шаги реализации:</h5>
                            <ol>{steps_html}</ol>
                        </div>
                        <div>
                            <h5>Ожидаемый эффект:</h5>
                            <p>{rec.get('expected_impact', '')}</p>

                            <h5 style="margin-top: 20px;">Решает проблемы:</h5>
                            <ul>{problems_html}</ul>
                        </div>
                    </div>

                    <div style="display: flex; gap: 15px; flex-wrap: wrap;">
                        <span class="tag" style="background: {self.colors['primary']}; color: white;">
                            {rec.get('timeline', 'Не определено')}
                        </span>
                        <span class="tag">Ресурсы: {rec.get('resources_needed', 'Не определено')}</span>
                        {metrics_tags}
                    </div>

                    {quote_html}
                </div>
                '''

        # Стратегические инициативы
        if recommendations.get('strategic_initiatives'):
            recs_html += '<h3 style="margin-top: 50px;">Стратегические инициативы</h3>'
            for i, init in enumerate(recommendations['strategic_initiatives'][:3], 1):
                # Фазы реализации
                phases_html = ''
                if init.get('implementation_phases'):
                    phases_list = ''.join([f'<li>{phase}</li>' for phase in init.get('implementation_phases', [])])
                    phases_html = f'<div style="margin: 20px 0;"><h5>Фазы реализации:</h5><ul>{phases_list}</ul></div>'

                recs_html += f'''
                <div class="card" style="border-top: 4px solid {self.colors['secondary']};">
                    <h4>{init.get('title', '')}</h4>
                    <p style="font-size: 1.1em; line-height: 1.8;">
                        {init.get('description', '')}
                    </p>

                    <div style="margin: 20px 0;">
                        <h5>Обоснование:</h5>
                        <p>{init.get('rationale', '')}</p>
                    </div>

                    {phases_html}

                    <div style="display: flex; gap: 15px;">
                        <span class="tag">ROI: {init.get('expected_roi', 'Требует оценки')}</span>
                    </div>
                </div>
                '''

        # Инновационные возможности
        if recommendations.get('innovation_opportunities'):
            recs_html += '<h3 style="margin-top: 50px;">Инновационные возможности</h3>'
            for opp in recommendations['innovation_opportunities'][:2]:
                # Требуемые исследования
                research_html = ''
                if opp.get('required_research'):
                    research_html = f'<p><strong>Требуется исследование:</strong> {opp.get("required_research", "")}</p>'

                recs_html += f'''
                <div class="card" style="background: linear-gradient(135deg, rgba(124, 58, 237, 0.05) 0%, rgba(37, 99, 235, 0.05) 100%);">
                    <h4>{opp.get('title', '')}</h4>
                    <p>{opp.get('description', '')}</p>
                    <p><strong>Потенциальное влияние:</strong> {opp.get('potential_impact', '')}</p>
                    {research_html}
                </div>
                '''

        return f'''
        <div class="page" id="recommendations">
            <div class="container">
                <h2>Рекомендации</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Конкретные действия для улучшения пользовательского опыта
                </p>

                {recs_html}
            </div>
        </div>
        '''

    def _generate_priority_matrix_section(self, recommendations):
        """Генерация матрицы приоритетов"""
        # Создаем простую текстовую матрицу
        matrix_html = '''
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 30px;">
            <div class="card" style="background: rgba(16, 185, 129, 0.05); border: 2px solid ''' + self.colors['success'] + ''';">
                <h4 style="color: ''' + self.colors['success'] + ''';">Высокое влияние / Низкие усилия</h4>
                <p style="font-size: 0.9em; color: ''' + self.colors['text_secondary'] + '''; margin-bottom: 20px;">
                    Приоритет: КРИТИЧЕСКИЙ - Начать немедленно
                </p>
        '''

        # Quick wins в эту категорию
        if recommendations.get('quick_wins'):
            for rec in recommendations['quick_wins'][:3]:
                matrix_html += f'<div style="margin: 10px 0; padding: 10px; background: white; border-radius: 8px;"><strong>• {rec.get("title", "")}</strong></div>'

        matrix_html += '''
            </div>
            <div class="card" style="background: rgba(37, 99, 235, 0.05); border: 2px solid ''' + self.colors['primary'] + ''';">
                <h4 style="color: ''' + self.colors['primary'] + ''';">Высокое влияние / Высокие усилия</h4>
                <p style="font-size: 0.9em; color: ''' + self.colors['text_secondary'] + '''; margin-bottom: 20px;">
                    Приоритет: ВЫСОКИЙ - Планировать на квартал
                </p>
        '''

        # Strategic initiatives в эту категорию
        if recommendations.get('strategic_initiatives'):
            for init in recommendations['strategic_initiatives'][:3]:
                matrix_html += f'<div style="margin: 10px 0; padding: 10px; background: white; border-radius: 8px;"><strong>• {init.get("title", "")}</strong></div>'

        matrix_html += '''
            </div>
            <div class="card" style="background: rgba(245, 158, 11, 0.05); border: 2px solid ''' + self.colors['warning'] + ''';">
                <h4 style="color: ''' + self.colors['warning'] + ''';">Низкое влияние / Низкие усилия</h4>
                <p style="font-size: 0.9em; color: ''' + self.colors['text_secondary'] + '''; margin-bottom: 20px;">
                    Приоритет: СРЕДНИЙ - Делегировать команде
                </p>
                <p style="font-style: italic;">Мелкие улучшения UX</p>
            </div>
            <div class="card" style="background: rgba(239, 68, 68, 0.05); border: 2px solid ''' + self.colors['danger'] + ''';">
                <h4 style="color: ''' + self.colors['danger'] + ''';">Низкое влияние / Высокие усилия</h4>
                <p style="font-size: 0.9em; color: ''' + self.colors['text_secondary'] + '''; margin-bottom: 20px;">
                    Приоритет: НИЗКИЙ - Переосмыслить или отложить
                </p>
                <p style="font-style: italic;">Полный редизайн без clear ROI</p>
            </div>
        </div>
        '''

        return f'''
        <div class="page" id="priority-matrix">
            <div class="container">
                <h2>Матрица приоритетов</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Распределение рекомендаций по влиянию и сложности реализации
                </p>

                {matrix_html}

                <div class="card" style="margin-top: 40px; background: {self.colors['background']};">
                    <h4>Рекомендуемый порядок действий:</h4>
                    <ol style="font-size: 1.1em; line-height: 2;">
                        <li><strong>Неделя 1-2:</strong> Запустить 2-3 quick wins для быстрых результатов</li>
                        <li><strong>Неделя 3-4:</strong> Провести детальное планирование стратегических инициатив</li>
                        <li><strong>Месяц 2:</strong> Начать реализацию первой стратегической инициативы</li>
                        <li><strong>Месяц 3:</strong> Оценить результаты и скорректировать план</li>
                    </ol>
                </div>
            </div>
        </div>
        '''

    def _generate_roadmap_section(self, defense):
        """Генерация дорожной карты"""
        next_steps = defense.get('next_steps', [])

        if not next_steps:
            return ''

        steps_html = ''
        for i, step in enumerate(next_steps[:6], 1):
            if isinstance(step, dict):
                steps_html += f'''
                <div class="roadmap-phase">
                    <div class="phase-number">{i}</div>
                    <h4 style="margin-left: 40px;">{step.get('action', '')}</h4>

                    <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin: 20px 0;">
                        <div>
                            <p><strong>Сроки:</strong> {step.get('timeline', '')}</p>
                            <p><strong>Ответственный:</strong> {step.get('responsible', '')}</p>
                        </div>
                        <div>
                            <p><strong>Ресурсы:</strong> {step.get('resources', '')}</p>
                            <p><strong>Ожидаемый результат:</strong> {step.get('expected_result', '')}</p>
                        </div>
                    </div>
                </div>
                '''
            else:
                steps_html += f'''
                <div class="roadmap-phase">
                    <div class="phase-number">{i}</div>
                    <p style="margin-left: 40px;">{step}</p>
                </div>
                '''

        # ROI расчеты
        roi_html = ''
        if defense.get('roi_calculation'):
            roi_html = f'''
            <div class="card" style="background: linear-gradient(135deg, {self.colors['background']} 0%, rgba(16, 185, 129, 0.05) 100%);">
                <h3>Расчет ROI</h3>
                <p style="font-size: 1.1em; line-height: 1.8;">
                    {defense['roi_calculation']}
                </p>
            </div>
            '''

        # Риски
        risks_html = ''
        if defense.get('risk_mitigation'):
            risks_html = '<h3>Управление рисками</h3>'
            for risk in defense['risk_mitigation']:
                if isinstance(risk, dict):
                    risks_html += f'''
                    <div class="card" style="border-left: 4px solid {self.colors['warning']};">
                        <h4>{risk.get('risk', '')}</h4>
                        <p><strong>Митигация:</strong> {risk.get('mitigation', '')}</p>
                        <p><strong>Мониторинг:</strong> {risk.get('monitoring', '')}</p>
                    </div>
                    '''

        # Метрики для отслеживания - исправляем проблемную f-строку
        metrics_html = ''
        if defense.get('success_metrics'):
            metrics_list = ''.join([f'<li>{metric}</li>' for metric in defense['success_metrics']])
        else:
            metrics_list = '<li>NPS Score</li><li>Количество критических проблем</li><li>Время на выполнение ключевых задач</li>'

        return f'''
        <div class="page" id="roadmap">
            <div class="container">
                <h2>Дорожная карта внедрения</h2>
                <p style="font-size: 1.1em; color: {self.colors['text_secondary']}; margin-bottom: 30px;">
                    Пошаговый план реализации рекомендаций
                </p>

                {steps_html}
                {roi_html}
                {risks_html}

                <div class="card" style="margin-top: 40px; background: {self.colors['primary']}; color: white;">
                    <h3 style="color: white;">Ключевые метрики для отслеживания</h3>
                    <ul style="font-size: 1.1em;">
                        {metrics_list}
                    </ul>
                </div>
            </div>
        </div>
        '''

    def _generate_detailed_appendix(self, summaries, analysis_data):
        """Генерация детального приложения"""
        appendix_html = '<h3>Детальная информация по интервью</h3>'

        for summary in summaries[:10]:  # Ограничиваем для читаемости
            profile = summary.respondent_profile

            # Ключевые темы
            themes_html = ''
            if summary.key_themes:
                themes_html = '<h5>Ключевые темы:</h5><ul>'
                for theme in summary.key_themes[:3]:
                    if isinstance(theme, dict):
                        themes_html += f'<li><strong>{theme.get("theme", "")}:</strong> {theme.get("description", "")[:150]}...</li>'
                themes_html += '</ul>'

            # Основные боли
            pains_html = ''
            if summary.pain_points:
                pains_html = '<h5>Основные проблемы:</h5><ul>'
                for pain in summary.pain_points[:3]:
                    if isinstance(pain, dict):
                        pains_html += f'<li>{pain.get("pain", "")[:150]}...</li>'
                pains_html += '</ul>'

            # Ключевые инсайты
            insights_html = ''
            if summary.insights:
                insights_html = '<h5>Инсайты:</h5><ul>'
                for insight in summary.insights[:3]:
                    if isinstance(insight, str):
                        insights_html += f'<li>{insight[:150]}...</li>'
                insights_html += '</ul>'

            appendix_html += f'''
            <div class="card" style="margin-bottom: 30px;">
                <h4>Интервью #{summary.interview_id}</h4>

                <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin: 20px 0;">
                    <div>
                        <p><strong>Демография:</strong> {profile.get('demographics', 'Не указано')}</p>
                        <p><strong>Профессия:</strong> {profile.get('occupation', 'Не указано')}</p>
                        <p><strong>Опыт:</strong> {profile.get('experience_level', 'Не указано')}</p>
                    </div>
                    <div>
                        <p><strong>Контекст:</strong> {profile.get('context', 'Не указано')[:100]}...</p>
                        <p><strong>Sentiment Score:</strong> {summary.sentiment_score:.2f}</p>
                    </div>
                </div>

                {themes_html}
                {pains_html}
                {insights_html}
            </div>
            '''

        # Методология
        methodology_html = f'''
        <h3 style="margin-top: 50px;">Методология исследования</h3>
        <div class="card">
            <h4>Сбор данных</h4>
            <ul>
                <li>Проведено <strong>{analysis_data.get('total_interviews', 0)}</strong> глубинных интервью</li>
                <li>Использован полуструктурированный гайд</li>
                <li>Средняя продолжительность интервью: 45-60 минут</li>
            </ul>

            <h4 style="margin-top: 20px;">Анализ данных</h4>
            <ul>
                <li>AI-ассистированный тематический анализ с Gemini</li>
                <li>Кодирование с фокусом на точные цитаты (минимум 50 слов)</li>
                <li>Кросс-валидация паттернов между интервью</li>
                <li>Приоритизация по частоте и критичности</li>
            </ul>

            <h4 style="margin-top: 20px;">Ограничения исследования</h4>
            <ul>
                <li>Размер выборки: {analysis_data.get('total_interviews', 0)} респондентов</li>
                {f'<li style="color: {self.colors["warning"]};">Выборка меньше рекомендуемой (8+ интервью)</li>' if analysis_data.get('total_interviews', 0) < 8 else ''}
                <li>Качественный характер данных</li>
                <li>Возможная субъективность интерпретаций</li>
            </ul>
        </div>
        '''

        return f'''
        <div class="page" id="appendix">
            <div class="container">
                <h2>Приложение</h2>

                {appendix_html}
                {methodology_html}

                <div class="card" style="margin-top: 40px; background: {self.colors['background']};">
                    <h4>Контактная информация</h4>
                    <p><strong>Подготовлено:</strong> {self.config.author}</p>
                    <p><strong>Организация:</strong> {self.config.name}</p>
                    <p><strong>Дата:</strong> {datetime.now().strftime("%d.%m.%Y")}</p>
                </div>
            </div>
        </div>
        '''

    def _generate_static_charts(self, analysis_data):
        """Генерация статичных графиков в base64"""
        charts = {}

        try:
            # График распределения проблем по severity
            problems = analysis_data.get('base_analysis', {}).get('problems', [])
            if problems:
                severity_counts = {}
                for p in problems:
                    sev = p.get('severity', 'medium')
                    severity_counts[sev] = severity_counts.get(sev, 0) + 1

                if severity_counts:
                    plt.figure(figsize=(8, 6))
                    colors_map = {
                        'critical': self.colors['danger'],
                        'high': self.colors['warning'],
                        'medium': self.colors['primary'],
                        'low': self.colors['success']
                    }

                    severities = list(severity_counts.keys())
                    counts = list(severity_counts.values())
                    colors = [colors_map.get(s, self.colors['primary']) for s in severities]

                    plt.bar(severities, counts, color=colors)
                    plt.title('Распределение проблем по критичности', fontsize=16, pad=20)
                    plt.xlabel('Критичность', fontsize=12)
                    plt.ylabel('Количество проблем', fontsize=12)

                    buffer = BytesIO()
                    plt.tight_layout()
                    plt.savefig(buffer, format='png', dpi=150, bbox_inches='tight', facecolor='white')
                    buffer.seek(0)
                    image_base64 = base64.b64encode(buffer.getvalue()).decode()
                    plt.close()

                    charts['problems_chart'] = f'''
                    <div class="chart-container">
                        <img src="data:image/png;base64,{image_base64}" style="width: 100%; max-width: 800px; margin: 0 auto; display: block;">
                    </div>
                    '''

            # График сегментов
            segments = analysis_data.get('base_analysis', {}).get('segments', [])
            if segments:
                plt.figure(figsize=(10, 6))
                segment_names = [s.get('name', f'Сегмент {i+1}')[:20] for i, s in enumerate(segments)]
                segment_sizes = []

                for s in segments:
                    size_str = s.get('size', '0%')
                    # Извлекаем число из строки типа "30-40%" или "3-4 из 8"
                    import re
                    match = re.search(r'(\d+)', size_str)
                    if match:
                        segment_sizes.append(int(match.group(1)))
                    else:
                        segment_sizes.append(20)  # default

                if segment_sizes:
                    colors = plt.cm.Set3(range(len(segments)))
                    plt.pie(segment_sizes, labels=segment_names, autopct='%1.0f%%', colors=colors, startangle=90)
                    plt.title('Распределение пользователей по сегментам', fontsize=16, pad=20)

                    buffer = BytesIO()
                    plt.tight_layout()
                    plt.savefig(buffer, format='png', dpi=150, bbox_inches='tight', facecolor='white')
                    buffer.seek(0)
                    image_base64 = base64.b64encode(buffer.getvalue()).decode()
                    plt.close()

                    charts['segments_chart'] = f'''
                    <div class="chart-container">
                        <img src="data:image/png;base64,{image_base64}" style="width: 100%; max-width: 600px; margin: 0 auto; display: block;">
                    </div>
                    '''

        except Exception as e:
            logging.error(f"Ошибка при генерации графиков: {e}")

        return charts

    def generate_docx(self, analysis_data):
        """Генерация DOCX отчета"""
        doc = Document()

        # Настройка стилей
        self._setup_docx_styles(doc)

        # Заголовок
        doc.add_heading(self.config.report_title, 0)
        doc.add_paragraph(f'{self.config.name}\n{datetime.now().strftime("%d.%m.%Y")}')
        doc.add_page_break()

        # Оглавление
        doc.add_heading('Содержание', level=1)
        doc.add_paragraph('1. Общий обзор')
        doc.add_paragraph('2. Ключевые находки')
        doc.add_paragraph('3. Сегменты пользователей')
        doc.add_paragraph('4. Персоны')
        doc.add_paragraph('5. Ключевые проблемы')
        doc.add_paragraph('6. Рекомендации')
        doc.add_paragraph('7. Дорожная карта')
        doc.add_paragraph('8. Приложение')
        doc.add_page_break()

        # Общий обзор
        self._add_overview_to_docx(doc, analysis_data)

        # Ключевые находки
        self._add_key_findings_to_docx(doc, analysis_data)

        # И так далее для всех разделов...

        return doc

    def _setup_docx_styles(self, doc):
        """Настройка стилей для DOCX"""
        # Настройка стиля заголовков
        styles = doc.styles

        # Heading 1
        heading1 = styles['Heading 1']
        heading1.font.name = 'Arial'
        heading1.font.size = Pt(24)
        heading1.font.color.rgb = RGBColor(37, 99, 235)

        # Heading 2
        heading2 = styles['Heading 2']
        heading2.font.name = 'Arial'
        heading2.font.size = Pt(18)
        heading2.font.color.rgb = RGBColor(37, 99, 235)

    def _add_overview_to_docx(self, doc, analysis_data):
        """Добавление обзора в DOCX"""
        doc.add_heading('Общий обзор', level=1)

        defense = analysis_data.get('defense_materials', {})
        exec_summary = defense.get('executive_summary', '')

        doc.add_paragraph(exec_summary)

        # Добавляем таблицу с метриками
        table = doc.add_table(rows=2, cols=4)
        table.style = 'Light Grid Accent 1'

        # Заголовки
        cells = table.rows[0].cells
        cells[0].text = 'Интервью'
        cells[1].text = 'Проблем'
        cells[2].text = 'Сегментов'
        cells[3].text = 'Quick Wins'

        # Данные
        cells = table.rows[1].cells
        cells[0].text = str(analysis_data.get('total_interviews', 0))
        cells[1].text = str(len(analysis_data.get('base_analysis', {}).get('problems', [])))
        cells[2].text = str(len(analysis_data.get('base_analysis', {}).get('segments', [])))
        cells[3].text = str(len(analysis_data.get('recommendations', {}).get('quick_wins', [])))

        doc.add_page_break()

    def _add_key_findings_to_docx(self, doc, analysis_data):
        """Добавление ключевых находок в DOCX"""
        doc.add_heading('Ключевые находки', level=1)

        defense = analysis_data.get('defense_materials', {})
        key_findings = defense.get('key_findings', [])

        for i, finding in enumerate(key_findings[:5], 1):
            doc.add_heading(f'Находка #{i}', level=2)
            doc.add_paragraph(finding)

    def generate_pdf(self, html_content):
        """Генерация PDF из HTML"""
        try:
            # Конвертируем HTML в PDF
            pdf = WeasyHTML(string=html_content).write_pdf()
            return pdf
        except Exception as e:
            logging.error(f"Ошибка при генерации PDF: {e}")
            return None

# ========================================================================
# ОСНОВНОЙ ИНТЕРФЕЙС
# ========================================================================
class UXAnalyzerInterface:
    def __init__(self):
        self.api_key = None
        self.analyzer = None
        self.transcripts = []
        self.brief_content = None
        self.company_config = CompanyConfig()
        self.output_widget = widgets.Output()
        self.progress_output = widgets.Output()

    def create_interface(self):
        """Создание интерфейса"""
        # Виджеты
        self.api_key_input = widgets.Password(
            placeholder='Введите ваш Gemini API ключ',
            description='API Key:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )

        self.company_name_input = widgets.Text(
            value='Company',
            placeholder='Название компании',
            description='Компания:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )

        self.report_title_input = widgets.Text(
            value='UX Research Report',
            placeholder='Название отчета',
            description='Название:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )

        self.author_input = widgets.Text(
            value='Research Team',
            placeholder='Автор отчета',
            description='Автор:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )

        self.upload_btn = widgets.Button(
            description='📁 Загрузить транскрипты',
            button_style='primary',
            layout=widgets.Layout(width='200px', height='40px')
        )

        self.upload_brief_btn = widgets.Button(
            description='📋 Загрузить бриф',
            button_style='info',
            layout=widgets.Layout(width='200px', height='40px')
        )

        self.analyze_btn = widgets.Button(
            description='🚀 Начать анализ',
            button_style='success',
            layout=widgets.Layout(width='200px', height='40px'),
            disabled=True
        )

        self.files_label = widgets.HTML(value='<p>Файлы не загружены</p>')
        self.brief_label = widgets.HTML(value='<p>Бриф не загружен (опционально)</p>')

        # Обработчики
        self.upload_btn.on_click(self._on_upload_click)
        self.upload_brief_btn.on_click(self._on_upload_brief_click)
        self.analyze_btn.on_click(self._on_analyze_click)

        # Компоновка
        config_box = widgets.VBox([
            widgets.HTML('<h2>⚙️ Настройки</h2>'),
            self.api_key_input,
            self.company_name_input,
            self.report_title_input,
            self.author_input
        ])

        upload_box = widgets.VBox([
            widgets.HTML('<h2>📤 Загрузка данных</h2>'),
            widgets.HBox([self.upload_btn, self.upload_brief_btn]),
            self.files_label,
            self.brief_label
        ])

        analyze_box = widgets.VBox([
            widgets.HTML('<h2>🔬 Анализ</h2>'),
            self.analyze_btn,
            self.progress_output
        ])

        main_layout = widgets.VBox([
            config_box,
            widgets.HTML('<hr>'),
            upload_box,
            widgets.HTML('<hr>'),
            analyze_box,
            widgets.HTML('<hr>'),
            self.output_widget
        ])

        display(main_layout)

    def _on_upload_click(self, b):
        """Обработчик загрузки транскриптов"""
        with self.output_widget:
            clear_output()
            print("📁 Выберите файлы с транскриптами...")

        uploaded = files.upload()

        if uploaded:
            self.transcripts = []
            for filename, content in uploaded.items():
                try:
                    if filename.endswith('.txt'):
                        text = content.decode('utf-8')
                    elif filename.endswith('.docx'):
                        doc = Document(BytesIO(content))
                        text = '\n'.join([p.text for p in doc.paragraphs])
                    else:
                        continue

                    self.transcripts.append(text)
                except Exception as e:
                    print(f"❌ Ошибка при чтении {filename}: {e}")

            self.files_label.value = f'<p>✅ Загружено транскриптов: {len(self.transcripts)}</p>'

            if len(self.transcripts) < 8:
                self.files_label.value += f'<p style="color: orange;">⚠️ Рекомендуется минимум 8 интервью для качественного анализа</p>'

            if self.transcripts and self.api_key_input.value:
                self.analyze_btn.disabled = False

    def _on_upload_brief_click(self, b):
        """Обработчик загрузки брифа"""
        with self.output_widget:
            clear_output()
            print("📋 Выберите файл с брифом...")

        uploaded = files.upload()

        if uploaded:
            for filename, content in uploaded.items():
                try:
                    if filename.endswith('.txt'):
                        self.brief_content = content.decode('utf-8')
                    elif filename.endswith('.docx'):
                        doc = Document(BytesIO(content))
                        self.brief_content = '\n'.join([p.text for p in doc.paragraphs])

                    self.brief_label.value = f'<p>✅ Бриф загружен: {filename}</p>'
                    break
                except Exception as e:
                    print(f"❌ Ошибка при чтении брифа: {e}")

    def _on_analyze_click(self, b):
        """Обработчик запуска анализа"""
        with self.output_widget:
            clear_output()

        if not self.api_key_input.value:
            with self.output_widget:
                print("❌ Введите API ключ!")
            return

        if not self.transcripts:
            with self.output_widget:
                print("❌ Загрузите транскрипты!")
            return

        # Обновляем конфигурацию
        self.company_config.name = self.company_name_input.value
        self.company_config.report_title = self.report_title_input.value
        self.company_config.author = self.author_input.value

        # Запускаем анализ
        self.analyze_btn.disabled = True
        self.analyze_btn.description = '⏳ Анализ...'

        try:
            # Создаем анализатор
            self.analyzer = AdvancedGeminiAnalyzer(self.api_key_input.value)

            # Устанавливаем бриф если есть
            if self.brief_content:
                self.analyzer.set_brief(self.brief_content)

            with self.progress_output:
                clear_output()
                # Запускаем анализ
                results = self.analyzer.analyze_transcripts_parallel(self.transcripts)

            # Генерируем отчеты
            self._generate_reports(results)

        except Exception as e:
            with self.output_widget:
                print(f"❌ Ошибка при анализе: {e}")
                traceback.print_exc()
        finally:
            self.analyze_btn.disabled = False
            self.analyze_btn.description = '🚀 Начать анализ'

    def _generate_reports(self, results):
        """Генерация отчетов"""
        with self.output_widget:
            clear_output()
            print("📊 Генерация отчетов...")

        generator = EnhancedReportGeneratorFixed(self.company_config)

        # Определяем timestamp в начале
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        html_content = None

        # HTML
        try:
            html_content = generator.generate_html(results)
            html_filename = f'ux_report_{timestamp}.html'

            with open(html_filename, 'w', encoding='utf-8') as f:
                f.write(html_content)

            with self.output_widget:
                print(f"✅ HTML отчет создан: {html_filename}")
                display(FileLink(html_filename))
        except Exception as e:
            with self.output_widget:
                print(f"❌ Ошибка при создании HTML: {e}")
                import traceback
                traceback.print_exc()

        # PDF - только если есть html_content
        if html_content:
            try:
                pdf_content = generator.generate_pdf(html_content)
                if pdf_content:
                    pdf_filename = f'ux_report_{timestamp}.pdf'
                    with open(pdf_filename, 'wb') as f:
                        f.write(pdf_content)

                    with self.output_widget:
                        print(f"✅ PDF отчет создан: {pdf_filename}")
                        display(FileLink(pdf_filename))
            except Exception as e:
                with self.output_widget:
                    print(f"⚠️ Не удалось создать PDF: {e}")

        # DOCX
        try:
            doc = generator.generate_docx(results)
            docx_filename = f'ux_report_{timestamp}.docx'
            doc.save(docx_filename)

            with self.output_widget:
                print(f"✅ DOCX отчет создан: {docx_filename}")
                display(FileLink(docx_filename))
        except Exception as e:
            with self.output_widget:
                print(f"⚠️ Не удалось создать DOCX: {e}")

        with self.output_widget:
            print("\n🎉 Анализ завершен!")
            print(f"📊 Проанализировано интервью: {results.get('total_interviews', 0)}")
            print(f"🔍 Выявлено ключевых проблем: {len(results.get('base_analysis', {}).get('problems', []))}")
            print(f"👥 Сегментов пользователей: {len(results.get('base_analysis', {}).get('segments', []))}")
            print(f"💡 Рекомендаций: {len(results.get('recommendations', {}).get('quick_wins', []))}")


🚀 Проверка и установка зависимостей...
✅ Зависимости уже установлены


In [ ]:
#@title 🚀 Интерфейс анализатора { run: "auto", display-mode: "form" }

# Тут ваш код интерфейса
# Он будет скрыт, но интерфейс отобразится

# ЗАПУСК
# ========================================================================
if __name__ == "__main__":
    print("🚀 UX Анализатор V24.0 - Точный анализ с фокусом на бриф")
    print("=" * 70)

    interface = UXAnalyzerInterface()
    interface.create_interface()

🚀 UX Анализатор V24.0 - Точный анализ с фокусом на бриф
